In [2]:
# {
# 'pop':'income',
# 'lifeExp':'number_of_low_risk_lifestyle_apps',
# 'gdpPercap':'number_of_low_risk_loan_apps'
# 'pop':'number_of_high_risk_games',
# 'lifeExp':'length_of_employment',
# 'gdpPercap':'question_of_other_loan'
# 'pop':'age',
# 'lifeExp':'gender',
# 'gdpPercap':'average_balance_last_three_months'
# 'pop':'number_of_p2p_payment_received',
# 'lifeExp':'number_of_matched_phone_numbers',
# 'gdpPercap':'number_of_online_transactions_in_last_3_months'
# 'pop':'time_filling_the_apps',
# 'lifeExp':'number_of_low_risk_wifi_ssid',
# 'gdpPercap':'count_rejected_or_not_qualified_prev_application_by_device_id'
# 'pop':'count_prev_applications_distinct_phone_number_by_device_id',
# 'lifeExp':'count_rejected_or_not_qualified_prev_application_by_nik'
# }

In [2]:
import pandas as pd
import numpy as np
import json
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.model_selection import cross_val_score,StratifiedShuffleSplit,StratifiedKFold
from sklearn.metrics import roc_auc_score,precision_score,recall_score,confusion_matrix,f1_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from xgboost import XGBClassifier,plot_importance
from datetime import datetime, timedelta
import re
import ipywidgets as widgets
from ipywidgets import interact,interactive
import os,sys
import pickle
import warnings
warnings.filterwarnings('ignore')
autostats_lib = "/home/jupyter/artemis/autostats"
if autostats_lib not in sys.path:
    sys.path.append(autostats_lib)
# import preprocessing.woe_transform as tf
# import redashrefresh as rf
# import pandas_profiling
import time
#import datetime as dt
%load_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jupyter/workspace/automatic-pancake/data-jupyter-notebook@athena-179008.json"

In [4]:
%%bigquery F01new

with provision as ( select cast((JSON_EXTRACT_SCALAR(`provision`, '$.value')) AS NUMERIC) as `provision`, `id` from `vayu.indodana_athena_rates` ) ,
disbursements as (
select contractId, max(disbursedDate) disbursedDate
from vayu.indodana_athena_disbursements
where status in ('COMPLETED','APPROVED')
group by 1
)
,
user_virtual_account as (
select `userId`, `virtualAccountNumber`
from vayu.indodana_athena_virtual_accounts
where `virtualAccountBank` = 'BCA'
),
payment as (select
`contractId`,
`paymentProviderReceivedDate`,
`paymentProvider`,
`principal`,
`interest`,
`dueDate`,
row_number() over(partition by `contractId` order by `paymentProviderReceivedDate` asc) as `rank`
from vayu.indodana_athena_user_payments
where `paymentProvider` <> 'WAIVER'
),
     payment_detail AS (--dipisahkan antara pembayaran yg sudah terjadi dari awal bulan lalu hingga saat ini dengan pembayaran yg masih aktif

                          (SELECT C.`approvedDate`,
                                  c.`updatedAt`,
                                  C.`applicationOrderId` AS `Order_Id`,
                                  C.`id` AS `contract_Id`,
                                  C.`status`,
                                  C.`approvedLoanAmount`*`provision` AS `approvedLoanAmount`,
                                  C.`approvedLoanTenureInMonth`,
                                  `rank` AS `payment`,
                                  C.`lateFee`,
                                  C.`latePaymentCount`,
                                  P.`principal`,
                                  P.`principal` + P.`interest` AS `installment`,
                                  P.`principal` + P.`interest` AS `balanceplusinterest`,
                                  CASE
                                      WHEN date(timestamp_add(`paymentProviderReceivedDate`, INTERVAL 7 HOUR)) >= (date_trunc(CURRENT_DATE(), MONTH)) 
                                        THEN (date_trunc(CURRENT_DATE(), MONTH))
                                      ELSE  date(timestamp_add(`paymentProviderReceivedDate`, INTERVAL 7 HOUR))
                                  END AS `PaymentDate` /*untuk membedakan pembayaran yg lebih dari akhir bulan*/,
                                  date(`dueDate`) AS `dueDate`
                           FROM `vayu.indodana_athena_contracts` C
                           JOIN payment P ON C.`id` = P.`contractId`
                           JOIN provision pr ON C.`rateId` = pr.`id`
                           WHERE date(C.`approvedDate`) < date_trunc(CURRENT_DATE(), MONTH)
                            --  AND (`paymentProviderReceivedDate` + interval '7 hours')::date >= date_trunc('month', CURRENT_DATE - interval '1 month')
                             AND C.`status` IN ('ACTIVE',
                                                'FINISHED')
                             AND C.`applicationOrderId` NOT LIKE '%LOP-%')
                        UNION DISTINCT
                          (SELECT C.`approvedDate`,
                                  c.`updatedAt`,
                                  C.`applicationOrderId` AS `Order_Id`,
                                  C.`id` AS `Contract_Id`,
                                  C.`status`,
                                  C.`approvedLoanAmount`*`provision` AS `approvedLoanAmount`,
                                  C.`approvedLoanTenureInMonth`,
                                  7 AS `payment`,
                                  C.`lateFee`,
                                  C.`latePaymentCount`,
                                  C.`balance` AS `principal`,
                                  C.`principal` + C.`interest` AS `installment`,
                                  C.`balanceWithInterest` AS `balanceplusinterest`,
                                  date_trunc(CURRENT_DATE(), MONTH) AS `PaymentDate`/*semua pembayaran aktif diset tgl pembayarannya*/,
                                  date(C.`nextDueDate`) AS `dueDate`
                           FROM `vayu.indodana_athena_contracts` C
                           JOIN `provision` pr ON c.`rateId` = pr.`id`
                           WHERE date(C.`approvedDate`) < date_trunc(CURRENT_DATE(), MONTH)
                             AND C.`status` = 'ACTIVE'
                             AND C.`applicationOrderId` NOT LIKE '%LOP-%')),
datalastmonth AS
  (SELECT pd.*, --dpd s.d. akhir bulan
 CASE

     WHEN `PaymentDate` <= date_trunc(date_add(CURRENT_DATE(), interval -1 day), MONTH) THEN date_diff(`PaymentDate`,`dueDate`, DAY)
     ELSE date_diff(date_trunc(date_add(CURRENT_DATE(), interval -1 day), MONTH), `dueDate`, DAY)
 END AS `dpdLM`
   FROM payment_detail pd),
     lastmonthactive AS
  (SELECT Order_Id,
          Contract_Id,
          lateFee,
          latePaymentCount,
          CASE
              WHEN status = 'FINISHED' THEN 'ACTIVE'
              ELSE status
          END AS status,
          approvedLoanAmount,
          MIN(payment) AS payment,
          SUM(principal) AS principalpaidafter,
          SUM(balanceplusinterest) AS balanceplusinterest,
          MIN(PaymentDate) AS tgl
   FROM datalastmonth
   WHERE PaymentDate > date_add(date_trunc(CURRENT_DATE, MONTH), interval -1 day)
   GROUP BY 1,
            2,
            3,
            4,
            5,
            6)
            ,
     lastmonthfinished AS
  (SELECT Order_Id,
          Contract_Id,
          lateFee,
          latePaymentCount,
          status,
          approvedLoanAmount,
          MAX(payment) AS payment,
          0 AS principalpaidafter,
          0 AS balanceplusinterest,
          MAX(PaymentDate) AS tgl
   FROM datalastmonth
   WHERE Order_Id NOT IN
       (SELECT DISTINCT Order_Id
        FROM lastmonthactive)
   GROUP BY 1,
            2,
            3,
            4,
            5,
            6)
            ,
     `pefindo_data` AS (
                          (SELECT lmf.`Order_Id`,
                                  lmf.`Contract_Id`,
                                  lmf.`status`,
                                  lmf.`lateFee`,
                                  lmf.`latePaymentCount`,
                                  lmf.`approvedLoanAmount`,
                                  lmf.`principalpaidafter` AS `balance`,
                                  lmf.`balanceplusinterest`,
                                  `installment`,
                                  `tgl` AS `paidoffdate`,
                                  `dueDate` AS `tgl`,
                                  `dpdLM`,
                                  `approvedDate`,
                                  `updatedAt`,
                                  `approvedLoanTenureInMonth`
                           FROM `lastmonthfinished` lmf
                           JOIN `datalastmonth` dlm ON lmf.`Order_Id` = dlm.`Order_Id`
                           AND lmf.`payment` = dlm.`payment`)
                        UNION DISTINCT
                          (SELECT lma.`Order_Id`,
                                  lma.`Contract_Id`,
                                  lma.`status`,
                                  lma.`lateFee`,
                                  lma.`latePaymentCount`,
                                  lma.`approvedLoanAmount`,
                                  lma.`principalpaidafter` AS `balance`,
                                  lma.`balanceplusinterest`,
                                  `installment`,
                                  NULL AS `paidoffdate`,
                                  `dueDate` AS `tgl`,
                                  `dpdLM`,
                                  `approvedDate`,
                                  `updatedAt`,
                                  `approvedLoanTenureInMonth`
                           FROM `lastmonthactive` lma
                           JOIN `datalastmonth` dlm ON lma.`Order_Id` = dlm.`Order_Id`
                           AND lma.`payment` = dlm.`payment`))
                          ,       
   summary AS
  (SELECT '0301' AS `Kode_Jenis_Pelapor`,
          '900013' AS `Kode_Pelapor`,
          FORMAT_DATE('%Y%m', CURRENT_DATE()) AS `Tahun_Bulan_Data`,
          'F01' AS `Kode_Jenis_Fasilitas`,
          a.`orderId` AS `Nomor_Rekening_Fasilitas`,
          NULL AS `Nomor_Rekening_Lama_Fasilitas`,
          replace(coalesce(uv.`virtualAccountNumber`,a.`applicantIdNumber`),'+','') AS `Nomor_CIF_Debitur`,
          9 AS `Kode_Sifat_Kredit_atau_Pembiayaan`,
          '05' AS `Kode_Jenis_Kredit_atau_Pembiayaan`,
          '00' AS `Kode_Akad_Kredit_atau_Akad_Pembiayaan`,
          a.`orderId` AS `Nomor_Akad_Awal`,
          FORMAT_DATE('%Y%m%d', date(`approvedDate`)) AS `Tanggal_Akad_Awal`,
          NULL AS `Nomor_Akad_Akhir`,
          NULL AS `Tanggal_Akad_Akhir`,
          0 AS `Frekuensi_Perpanjangan_Fasilitas_Kredit_atau_Pembiayaan`,
          FORMAT_DATE('%Y%m%d', date(`approvedDate`)) AS `Tanggal_Awal_Kredit_atau_Pembiayaan`,
          FORMAT_DATE('%Y%m%d', date(IC.`startDate`)) AS `Tanggal_Mulai`,
          FORMAT_DATE('%Y%m%d', date_add(date(cast(EXTRACT(YEAR FROM `startDate`) as INT64),CAST(EXTRACT(MONTH FROM `startDate`) as INT64),`day`), interval `approvedLoanTenureInMonth` Month)) AS `Tanggal_Jatuh_Tempo`,
          99 AS `Kode_Kategori_Debitur`,
          CASE
              WHEN A.`loanPurpose` = 'PENGEMBANGAN USAHA' THEN 2
              ELSE 3
          END AS `Kode_Jenis_Penggunaan`,
          3 AS `Kode_Orientasi_Penggunaan`,
          '009000' AS `Kode_Sektor_Ekonomi`,
          CASE
              WHEN A.`addressIsCurrentAddress` = 'Ya' THEN A.`applicantResidenceCity`
              ELSE A.`applicantCurrentResidenceCity`
          END AS `City`,
          CASE
              WHEN a.`loanPurpose` = 'PENGEMBANGAN USAHA' THEN `loanAmount`
              ELSE NULL
          END AS `Nilai_Proyek`,
          'IDR' AS `Kode_Valuta`,
          0.96 AS `Suku_Bunga_atau_Imbalan`,
          1 AS `Jenis_Suku_Bunga_atau_Imbalan`,
          '001' AS `Kredit_atau_Pembiayaan_Program_Pemerintah`,
          pd.`approvedLoanAmount` AS `Plafon_Awal`,
          pd.`approvedLoanAmount` AS `Plafon`,
          pd.`approvedLoanAmount` AS `Realisasi_atau_Pencairan_Bulan_Berjalan`,
          `lateFee` `Denda`,
          `balance` AS `Baki_Debet`,
          NULL AS `Nilai_dalam_Mata_Uang_Asal`,
          CASE
              WHEN `dpdLM` < 1 THEN 1
              WHEN `dpdLM` <= 90 THEN 2
              WHEN `dpdLM` <= 120 THEN 3
              WHEN `dpdLM` <= 180 THEN 4
              WHEN `dpdLM` > 180 THEN 5
              ELSE 1
          END AS `Kode_Kualitas_Kredit_atau_Pembiayaan`,
          CASE
              WHEN `dpdLM` > 180 THEN FORMAT_DATE('%Y%m%d', date_add(`tgl`, interval 180 DAY))
              ELSE NULL
          END AS `Tanggal_Macet`,
          CASE
              WHEN `dpdLM` > 180 THEN 99
              ELSE NULL
          END AS `Kode_Sebab_Macet`,
          pd.`balance` AS `Tunggakan_Pokok`,
          0 AS `Tunggakan_Bunga_atau_Imbalan`,
          CASE
              WHEN `dpdLM` > 0 THEN `dpdLM`
              ELSE 0
          END AS `Jumlah_Hari_Tunggakan`,
          `latePaymentCount` AS `Frekuensi_Tunggakan`,
          0 AS `Frekuensi_Restrukturisasi`,
          '' AS `Tanggal_Restrukturisasi_Awal`,
          '' AS `Tanggal_Restrukturisasi_Akhir`,
          '' AS `Kode_Cara_Restrukturisasi`,
          CASE
              WHEN pd.`status` IN ('WAITING_FOR_DISBURSEMENT',
                                  'ACTIVE') THEN '00'
              WHEN pd.`status` = 'FINISHED' THEN '02'
          END AS `Kode_Kondisi`,
          CASE
              WHEN pd.`status` NOT IN ('WAITING_FOR_DISBURSEMENT',
                                      'ACTIVE') THEN pd.`updatedAt`
              ELSE NULL
          END AS `Tanggal_Kondisi`,
          NULL AS `Keterangan`,
          '000' AS `Kode_Kantor_Cabang`,
          'C' AS `Operasi_Data`,
          'T' AS `Status_delete`,
          FORMAT_TIMESTAMP('%Y%m%d%H%M%S', `approvedDate`) AS `Create_Date`,
          FORMAT_TIMESTAMP('%Y%m%d%H%M%S', pd.`updatedAt`) AS `Update_Date`
  FROM pefindo_data pd
  INNER JOIN vayu.indodana_athena_applications A ON pd.`Order_Id` = A.`orderId`
  INNER JOIN disbursements D ON pd.`Contract_Id` = D.`contractId`
  INNER JOIN vayu.indodana_athena_installment_commitments ic ON pd.`Contract_Id` = ic.`contractId`
  INNER JOIN user_virtual_account uv ON A.`userId` = uv.`userId`
   UNION DISTINCT SELECT '0301' AS `Kode_Jenis_Pelapor`,
                '900013' AS `Kode_Pelapor`,
                FORMAT_DATE('%Y%m',CURRENT_DATE()) AS `Tahun_Bulan_Data`,
                'F01' AS `Kode_Jenis_Fasilitas`,
                a.`orderId` AS `Nomor_Rekening_Fasilitas`,
                NULL AS `Nomor_Rekening_Lama_Fasilitas`,
                replace(coalesce(uv.`virtualAccountNumber`,a.`applicantIdNumber`),'+','') AS `Nomor_CIF_Debitur`,
                9 AS `Kode Sifat Kredit_atau_Pembiayaan`,
                '05' AS `Kode_Jenis_Kredit_atau_Pembiayaan`,
                '00' AS `Kode Akad Kredit atau Akad Pembiayaan`,
                a.`orderId` AS `Nomor_Akad_Awal`,
                FORMAT_DATE('%Y%m%d',date(`approvedDate`)) AS `Tanggal Akad Awal`,
                NULL AS `Nomor_Akad_Akhir`,
                NULL AS `Tanggal_Akad_Akhir`,
                0 AS `Frekuensi_Perpanjangan_Fasilitas_Kredit_atau_Pembiayaan`,
                FORMAT_DATE('%Y%m%d', date(C.`approvedDate`)) AS `Tanggal_Awal_Kredit_atau_Pembiayaan`,
                FORMAT_DATE('%Y%m%d', date(IC.`startDate`)) AS `Tanggal_Mulai`,
                FORMAT_DATE('%Y%m%d', date_add(date(cast(EXTRACT(YEAR FROM `startDate`) as INT64),CAST(EXTRACT(MONTH FROM `startDate`) as INT64),`day`), interval `approvedLoanTenureInMonth` Month)) AS `Tanggal_Jatuh_Tempo`,
                99 AS `Kode_Kategori_Debitur`,
                CASE
                    WHEN A.`loanPurpose` = 'PENGEMBANGAN USAHA' THEN 2
                    ELSE 3
                END AS `Kode_Jenis_Penggunaan`,
                3 AS `Kode_Orientasi_Penggunaan`,
                '009000' AS `Kode_Sektor_Ekonomi`,
                CASE
                    WHEN A.`addressIsCurrentAddress` = 'Ya' THEN A.`applicantResidenceCity`
                    ELSE A.`applicantCurrentResidenceCity`
                END AS `City`,
                CASE
                    WHEN a.`loanPurpose` = 'PENGEMBANGAN USAHA' THEN `loanAmount`
                    ELSE NULL
                END AS `Nilai_Proyek`,
                'IDR' AS `Kode_Valuta`,
                0.96 AS `Suku_Bunga_atau_Imbalan`,
                1 AS `Jenis_Suku_Bunga_atau_Imbalan`,
                '001' AS `Kredit atau Pembiayaan Program Pemerintah`,
                C.`approvedLoanAmount`*P.`provision` AS `Plafon Awal`,
                C.`approvedLoanAmount`*P.`provision` AS `Plafon`,
                C.`approvedLoanAmount`*P.`provision` AS `Realisasi atau Pencairan Bulan Berjalan`,
                0 `Denda`,
                0 AS `Baki Debet`,
                NULL AS `Nilai_dalam_Mata_Uang_Asal`,
                1 AS `Kode_Kualitas_Kredit_atau_Pembiayaan `,
                NULL AS `Tanggal_Macet`,
                NULL AS `Kode_Sebab_Macet`,
                0 AS `Tunggakan_Pokok`,
                0 AS `Tunggakan_Bunga_atau_Imbalan`,
                0 AS `Jumlah_Hari_Tunggakan`,
                0 AS `Frekuensi_Tunggakan`,
                0 AS `Frekuensi_Restrukturisasi`,
                '' AS `Tanggal_Restrukturisasi_Awal`,
                '' AS `Tanggal_Restrukturisasi_Akhir`,
                '' AS `Kode_Cara_Restrukturisasi`,
                '01' AS `Kode_Kondisi`,
                c.`updatedAt` `Tanggal_Kondisi`,
                NULL AS `Keterangan`,
                '000' AS `Kode Kantor Cabang`,
                'C' AS `Operasi Data`,
                'T' AS `Status delete`,
                FORMAT_TIMESTAMP('%Y%m%d%H%M%S', (c.`createdAt`)) AS `Create Date`,
                FORMAT_TIMESTAMP('%Y%m%d%H%M%S', (c.`updatedAt`)) AS `Update Date`
   FROM `vayu.indodana_athena_contracts` C
   JOIN `vayu.indodana_athena_applications` A ON C.`applicationOrderId` = A.`orderId`
   JOIN `vayu.indodana_athena_installment_commitments` IC ON C.`id` = IC.`contractId`
   JOIN `provision` P ON C.`rateId` = P.`id`
   JOIN `user_virtual_account` uv ON A.`userId` = uv.`userId`
   WHERE date(`approvedDate`) < date_trunc(CURRENT_DATE, MONTH)
    --  AND `approvedDate` >= date_trunc('month',CURRENT_DATE - interval '1 month')
     AND C.`status` IN ('REJECTED',
                        'CANCELLED') )                      

select
`Kode_Jenis_Pelapor`,
`Kode_Pelapor`,
`Tahun_Bulan_Data`,
`Kode_Jenis_Fasilitas`,
`Nomor_Rekening_Fasilitas`,
`Nomor_Rekening_Lama_Fasilitas`,
`Nomor_CIF_Debitur`,
`Kode_Sifat_Kredit_atau_Pembiayaan`,
`Kode_Jenis_Kredit_atau_Pembiayaan`,
`Kode_Akad_Kredit_atau_Akad_Pembiayaan`,
`Nomor_Akad_Awal`,
`Tanggal_Akad_Awal`,
`Nomor_Akad_Akhir`,
`Tanggal_Akad_Akhir`,
`Frekuensi_Perpanjangan_Fasilitas_Kredit_atau_Pembiayaan`,
`Tanggal_Awal_Kredit_atau_Pembiayaan`,
`Tanggal_Mulai`,
`Tanggal_Jatuh_Tempo`,
`Kode_Kategori_Debitur`,
`Kode_Jenis_Penggunaan`,
`Kode_Orientasi_Penggunaan`,
`Kode_Sektor_Ekonomi`,
case
WHEN`City` = 'PROVINSI JAWA BARAT' then '0100'
WHEN`City` = 'KABUPATEN BEKASI' then '0102'
WHEN`City` = 'KABUPATEN PURWAKARTA' then '0103'
WHEN`City` = 'KABUPATEN KARAWANG' then '0106'
WHEN`City` = 'KABUPATEN BOGOR' then '0108'
WHEN`City` = 'BOGOR' then '0108'
WHEN`City` = 'KABUPATEN SUKABUMI' then '0109'
WHEN`City` = 'KABUPATEN CIANJUR' then '0110'
WHEN`City` = 'KABUPATEN BANDUNG' then '0111'
WHEN`City` = 'KABUPATEN SUMEDANG' then '0112'
WHEN`City` = 'KABUPATEN TASIKMALAYA' then '0113'
WHEN`City` = 'KABUPATEN GARUT' then '0114'
WHEN`City` = 'KABUPATEN CIAMIS' then '0115'
WHEN`City` = 'KABUPATEN CIREBON' then '0116'
WHEN`City` = 'KABUPATEN KUNINGAN' then '0117'
WHEN`City` = 'KABUPATEN INDRAMAYU' then '0118'
WHEN`City` = 'KABUPATEN MAJALENGKA' then '0119'
WHEN`City` = 'KABUPATEN SUBANG' then '0121'
WHEN`City` = 'KABUPATEN BANDUNG BARAT' then '0122'
WHEN`City` = 'KOTA BANJAR' then '0180'
WHEN`City` = 'KOTA BANDUNG' then '0191'
WHEN`City` = 'KOTA BOGOR' then '0192'
WHEN`City` = 'KOTA SUKABUMI' then '0193'
WHEN`City` = 'KOTA CIREBON' then '0194'
WHEN`City` = 'KOTA TASIKMALAYA' then '0195'
WHEN`City` = 'KOTA CIMAHI' then '0196'
WHEN`City` = 'KOTA DEPOK' then '0197'
WHEN`City` = 'KOTA BEKASI' then '0198'
WHEN`City` = 'KABUPATEN PANGANDARAN' then '0123'
WHEN`City` = 'PROVINSI BANTEN' then '0200'
WHEN`City` = 'KABUPATEN LEBAK' then '0201'
WHEN`City` = 'KABUPATEN PANDEGLANG' then '0202'
WHEN`City` = 'KABUPATEN SERANG' then '0203'
WHEN`City` = 'KABUPATEN TANGERANG' then '0204'
WHEN`City` = 'KOTA CILEGON' then '0291'
WHEN`City` = 'KOTA TANGERANG' then '0292'
WHEN`City` = 'KOTA SERANG' then '0293'
WHEN`City` = 'KOTA TANGERANG SELATAN' then '0294'
WHEN`City` = 'KOTA BANTEN' then '0294'
WHEN`City` = 'PROVINSI DKI JAYA' then '0300'
WHEN`City` = 'KOTA JAKARTA PUSAT' then '0391'
WHEN`City` = 'KOTA JAKARTA UTARA' then '0392'
WHEN`City` = 'KOTA JAKARTA BARAT' then '0393'
WHEN`City` = 'KOTA JAKARTA SELATAN' then '0394'
WHEN`City` = 'KOTA ADM. JAKARTA SELATAN' then '0394'
WHEN`City` = 'KOTA JAKARTA TIMUR' then '0395'
WHEN`City` = 'KEPULAUAN SERIBU' then '0396'
WHEN`City` = 'KABUPATEN KEPULAUAN SERIBU' then '0396'
WHEN`City` = 'DAERAH ISTIMEWA YOGYAKARTA' then '0500'
WHEN`City` = 'KABUPATEN BANTUL' then '0501'
WHEN`City` = 'KABUPATEN SLEMAN' then '0502'
WHEN`City` = 'KABUPATEN GUNUNG KIDUL' then '0503'
WHEN`City` = 'KABUPATEN KULON PROGO' then '0504'
WHEN`City` = 'KOTA YOGYAKARTA' then '0591'
WHEN`City` = 'KABUPATEN PEKALONGAN' then '0905'
WHEN`City` = 'KABUPATEN TEGAL' then '0906'
WHEN`City` = 'KABUPATEN BREBES' then '0907'
WHEN`City` = 'KABUPATEN PATI' then '0908'
WHEN`City` = 'KABUPATEN KUDUS' then '0909'
WHEN`City` = 'KABUPATEN PEMALANG' then '0910'
WHEN`City` = 'KABUPATEN JEPARA' then '0911'
WHEN`City` = 'KABUPATEN REMBANG' then '0912'
WHEN`City` = 'KABUPATEN BLORA' then '0913'
WHEN`City` = 'KABUPATEN BANYUMAS' then '0914'
WHEN`City` = 'KABUPATEN CILACAP' then '0915'
WHEN`City` = 'KABUPATEN PURBALINGGA' then '0916'
WHEN`City` = 'KABUPATEN BANJARNEGARA' then '0917'
WHEN`City` = 'KABUPATEN MAGELANG' then '0918'
WHEN`City` = 'KABUPATEN TEMANGGUNG' then '0919'
WHEN`City` = 'KABUPATEN WONOSOBO' then '0920'
WHEN`City` = 'KABUPATEN PURWOREJO' then '0921'
WHEN`City` = 'KABUPATEN KEBUMEN' then '0922'
WHEN`City` = 'KABUPATEN KLATEN' then '0923'
WHEN`City` = 'KABUPATEN BOYOLALI' then '0924'
WHEN`City` = 'KABUPATEN SRAGEN' then '0925'
WHEN`City` = 'KABUPATEN SUKOHARJO' then '0926'
WHEN`City` = 'KABUPATEN KARANGANYAR' then '0927'
WHEN`City` = 'KABUPATEN WONOGIRI' then '0928'
WHEN`City` = 'KABUPATEN BATANG' then '0929'
WHEN`City` = 'KOTA SEMARANG' then '0991'
WHEN`City` = 'KOTA SALATIGA' then '0992'
WHEN`City` = 'KOTA PEKALONGAN' then '0993'
WHEN`City` = 'KOTA TEGAL' then '0994'
WHEN`City` = 'KOTA MAGELANG' then '0995'
WHEN`City` = 'KOTA SURAKARTA' then '0996'
WHEN`City` = 'KOTA SOLO' then '0996'
WHEN`City` = 'KABUPATEN GROBOGAN' then '0904'
WHEN`City` = 'PROVINSI JAWA TENGAH' then '0900'
WHEN`City` = 'KABUPATEN SEMARANG' then '0901'
WHEN`City` = 'KABUPATEN KENDAL' then '0902'
WHEN`City` = 'KABUPATEN DEMAK' then '0903'
WHEN`City` = 'PROVINSI JAWA TIMUR' then '1200'
WHEN`City` = 'KABUPATEN GRESIK' then '1201'
WHEN`City` = 'KABUPATEN SIDOARJO' then '1202'
WHEN`City` = 'KABUPATEN MOJOKERTO' then '1203'
WHEN`City` = 'KABUPATEN JOMBANG' then '1204'
WHEN`City` = 'KABUPATEN SAMPANG' then '1205'
WHEN`City` = 'KABUPATEN PAMEKASAN' then '1206'
WHEN`City` = 'KABUPATEN SUMENEP' then '1207'
WHEN`City` = 'KABUPATEN BANGKALAN' then '1208'
WHEN`City` = 'KABUPATEN BONDOWOSO' then '1209'
WHEN`City` = 'KABUPATEN BANYUWANGI' then '1211'
WHEN`City` = 'KABUPATEN JEMBER' then '1212'
WHEN`City` = 'KABUPATEN MALANG' then '1213'
WHEN`City` = 'KABUPATEN PASURUAN' then '1214'
WHEN`City` = 'KABUPATEN PROBOLINGGO' then '1215'
WHEN`City` = 'KABUPATEN LUMAJANG' then '1216'
WHEN`City` = 'KABUPATEN KEDIRI' then '1217'
WHEN`City` = 'KABUPATEN NGANJUK' then '1218'
WHEN`City` = 'KABUPATEN TULUNGAGUNG' then '1219'
WHEN`City` = 'KABUPATEN TRENGGALEK' then '1220'
WHEN`City` = 'KABUPATEN BLITAR' then '1221'
WHEN`City` = 'KABUPATEN MADIUN' then '1222'
WHEN`City` = 'KABUPATEN NGAWI' then '1223'
WHEN`City` = 'KABUPATEN MAGETAN' then '1224'
WHEN`City` = 'KABUPATEN PONOROGO' then '1225'
WHEN`City` = 'KABUPATEN PACITAN' then '1226'
WHEN`City` = 'KABUPATEN BOJONEGORO' then '1227'
WHEN`City` = 'KABUPATEN TUBAN' then '1228'
WHEN`City` = 'KABUPATEN LAMONGAN' then '1229'
WHEN`City` = 'KABUPATEN SITUBONDO' then '1230'
WHEN`City` = 'KOTA BATU' then '1271'
WHEN`City` = 'KOTA SURABAYA' then '1291'
WHEN`City` = 'KOTA MOJOKERTO' then '1292'
WHEN`City` = 'KOTA MALANG' then '1293'
WHEN`City` = 'KOTA PASURUAN' then '1294'
WHEN`City` = 'KOTA PROBOLINGGO' then '1295'
WHEN`City` = 'KOTA BLITAR' then '1296'
WHEN`City` = 'KOTA KEDIRI' then '1297'
WHEN`City` = 'KOTA MADIUN' then '1298'
WHEN`City` = 'PROVINSI BENGKULU' then '2300'
WHEN`City` = 'KABUPATEN BENGKULU SELATAN' then '2301'
WHEN`City` = 'KABUPATEN BENGKULU UTARA' then '2302'
WHEN`City` = 'KABUPATEN REJANG LEBONG' then '2303'
WHEN`City` = 'KABUPATEN LEBONG' then '2304'
WHEN`City` = 'KABUPATEN KEPAHIANG' then '2305'
WHEN`City` = 'KABUPATEN MUKOMUKO' then '2306'
WHEN`City` = 'KABUPATEN SELUMA' then '2307'
WHEN`City` = 'KABUPATEN KAUR' then '2308'
WHEN`City` = 'KABUPATEN BENGKULU TENGAH' then '2309'
WHEN`City` = 'KOTA BENGKULU' then '2391'
WHEN`City` = 'PROVINSI JAMBI' then '3100'
WHEN`City` = 'KABUPATEN BATANGHARI' then '3101'
WHEN`City` = 'KABUPATEN SAROLANGUN' then '3104'
WHEN`City` = 'KABUPATEN KERINCI' then '3105'
WHEN`City` = 'KABUPATEN MUARO JAMBI' then '3106'
WHEN`City` = 'KABUPATEN TANJUNG JABUNG BARAT' then '3107'
WHEN`City` = 'KABUPATEN TANJUNG JABUNG TIMUR' then '3108'
WHEN`City` = 'KABUPATEN TEBO' then '3109'
WHEN`City` = 'KABUPATEN MERANGIN' then '3111'
WHEN`City` = 'KABUPATEN BUNGO' then '3112'
WHEN`City` = 'KOTA JAMBI' then '3191'
WHEN`City` = 'KOTA SUNGAI PENUH' then '3192'
WHEN`City` = 'PROVINSI NAD' then '3200'
WHEN`City` = 'KABUPATEN ACEH BESAR' then '3201'
WHEN`City` = 'KABUPATEN PIDIE' then '3202'
WHEN`City` = 'KABUPATEN ACEH UTARA' then '3203'
WHEN`City` = 'KABUPATEN ACEH TIMUR' then '3204'
WHEN`City` = 'KABUPATEN ACEH SELATAN' then '3205'
WHEN`City` = 'KABUPATEN ACEH BARAT' then '3206'
WHEN`City` = 'KABUPATEN ACEH TENGAH' then '3207'
WHEN`City` = 'KABUPATEN ACEH TENGGARA' then '3208'
WHEN`City` = 'KABUPATEN ACEH SINGKIL' then '3209'
WHEN`City` = 'KABUPATEN ACEH JEUMPA /BIREUEN' then '3210'
WHEN`City` = 'KABUPATEN ACEH TAMIANG' then '3211'
WHEN`City` = 'KABUPATEN GAYO LUWES' then '3212'
WHEN`City` = 'KABUPATEN ACEH BARAT DAYA' then '3213'
WHEN`City` = 'KABUPATEN ACEH JAYA' then '3214'
WHEN`City` = 'KABUPATEN NAGAN RAYA' then '3215'
WHEN`City` = 'KABUPATEN SIMEULEU' then '3216'
WHEN`City` = 'KABUPATEN BENER MERIAH' then '3217'
WHEN`City` = 'KABUPATEN PIDIE JAYA' then '3218'
WHEN`City` = 'KABUPATEN SUBULUSSALAM' then '3219'
WHEN`City` = 'KOTA BANDA ACEH' then '3291'
WHEN`City` = 'KOTA SABANG' then '3292'
WHEN`City` = 'KOTA LHOKSEUMAWE' then '3293'
WHEN`City` = 'KOTA LANGSA' then '3294'
WHEN`City` = 'PROVINSI SUMATERA UTARA' then '3300'
WHEN`City` = 'KABUPATEN DELI SERDANG' then '3301'
WHEN`City` = 'KABUPATEN LANGKAT' then '3302'
WHEN`City` = 'KABUPATEN KARO' then '3303'
WHEN`City` = 'KABUPATEN SIMALUNGUN' then '3304'
WHEN`City` = 'KABUPATEN LABUHAN BATU' then '3305'
WHEN`City` = 'KABUPATEN ASAHAN' then '3306'
WHEN`City` = 'KABUPATEN DAIRI' then '3307'
WHEN`City` = 'KABUPATEN TAPANULI UTARA' then '3308'
WHEN`City` = 'KABUPATEN TAPANULI TENGAH' then '3309'
WHEN`City` = 'KABUPATEN TAPANULI SELATAN' then '3310'
WHEN`City` = 'KABUPATEN NIAS' then '3311'
WHEN`City` = 'KABUPATEN TOBA SAMOSIR' then '3313'
WHEN`City` = 'KABUPATEN MANDAILING NATAL' then '3314'
WHEN`City` = 'KABUPATEN NIAS SELATAN' then '3315'
WHEN`City` = 'KABUPATEN HUMBANG HASUNDUTAN' then '3316'
WHEN`City` = 'KABUPATEN PAKPAK BHARAT' then '3317'
WHEN`City` = 'KABUPATEN SAMOSIR' then '3318'
WHEN`City` = 'KABUPATEN SERDANG BEDAGAI' then '3319'
WHEN`City` = 'KABUPATEN BATU BARA' then '3321'
WHEN`City` = 'KABUPATEN PADANG LAWAS' then '3322'
WHEN`City` = 'KABUPATEN PADANG LAWAS UTARA' then '3323'
WHEN`City` = 'KABUPATEN LABUANBATU SELATAN' then '3324'
WHEN`City` = 'KABUPATEN LABUANBATU UTARA' then '3325'
WHEN`City` = 'KABUPATEN NIAS BARAT' then '3326'
WHEN`City` = 'KABUPATEN NIAS UTARA' then '3327'
WHEN`City` = 'KOTA TEBING TINGGI' then '3391'
WHEN`City` = 'KOTA BINJAI' then '3392'
WHEN`City` = 'KOTA PEMATANG SIANTAR' then '3393'
WHEN`City` = 'KOTA TANJUNG BALAI' then '3394'
WHEN`City` = 'KOTA SIBOLGA' then '3395'
WHEN`City` = 'KOTA MEDAN' then '3396'
WHEN`City` = 'KOTA GUNUNG SITOLI' then '3397'
WHEN`City` = 'KOTA PADANG SIDEMPUAN' then '3399'
WHEN`City` = 'PROVINSI SUMATERA BARAT' then '3400'
WHEN`City` = 'KABUPATEN AGAM' then '3401'
WHEN`City` = 'KABUPATEN PASAMAN' then '3402'
WHEN`City` = 'KABUPATEN LIMAPULUH KOTA' then '3403'
WHEN`City` = 'KABUPATEN SOLOK SELATAN' then '3404'
WHEN`City` = 'KABUPATEN PADANG PARIAMAN' then '3405'
WHEN`City` = 'KABUPATEN PESISIR SELATAN' then '3406'
WHEN`City` = 'KABUPATEN TANAH DATAR' then '3407'
WHEN`City` = 'KABUPATEN SAWAH LUNTO/SIJUNJUNG' then '3408'
WHEN`City` = 'KABUPATEN KEPULAUAN MENTAWAI' then '3409'
WHEN`City` = 'KABUPATEN PASAMAN BARAT' then '3410'
WHEN`City` = 'KABUPATEN DHARMASRAYA' then '3411'
WHEN`City` = 'KABUPATEN SOLOK' then '3412'
WHEN`City` = 'KOTA BUKITTINGGI' then '3491'
WHEN`City` = 'KOTA PADANG' then '3492'
WHEN`City` = 'KOTA SAWAHLUNTO' then '3493'
WHEN`City` = 'KOTA PADANG PANJANG' then '3494'
WHEN`City` = 'KOTA SOLOK' then '3495'
WHEN`City` = 'KOTA PAYAKUMBUH' then '3496'
WHEN`City` = 'KOTA PARIAMAN' then '3497'
WHEN`City` = 'PROVINSI RIAU' then '3500'
WHEN`City` = 'KABUPATEN KAMPAR' then '3501'
WHEN`City` = 'KABUPATEN BENGKALIS' then '3502'
WHEN`City` = 'KABUPATEN INDRAGIRI HULU' then '3504'
WHEN`City` = 'KABUPATEN INDRAGIRI HILIR' then '3505'
WHEN`City` = 'KABUPATEN ROKAN HULU' then '3508'
WHEN`City` = 'KABUPATEN ROKAN HILIR' then '3509'
WHEN`City` = 'KABUPATEN PELALAWAN' then '3510'
WHEN`City` = 'KABUPATEN SIAK' then '3511'
WHEN`City` = 'KABUPATEN KUANTAN SINGINGI' then '3512'
WHEN`City` = 'KABUPATEN KEPULAUAN MERANTI' then '3513'
WHEN`City` = 'KOTA PEKANBARU' then '3591'
WHEN`City` = 'KOTA DUMAI' then '3592'
WHEN`City` = 'PROVINSI SUMATERA SELATAN' then '3600'
WHEN`City` = 'KABUPATEN MUSI BANYUASIN' then '3606'
WHEN`City` = 'KABUPATEN OGAN KOMERING ULU' then '3607'
WHEN`City` = 'KABUPATEN MUARA ENIM' then '3608'
WHEN`City` = 'KABUPATEN LAHAT' then '3609'
WHEN`City` = 'KABUPATEN MUSI RAWAS' then '3610'
WHEN`City` = 'KABUPATEN OGAN KOMERING ILIR' then '3611'
WHEN`City` = 'KABUPATEN BANYUASIN' then '3613'
WHEN`City` = 'KABUPATEN OGAN KOMERING ULU SELATAN' then '3614'
WHEN`City` = 'KABUPATEN OGAN KOMERING ULU TIMUR' then '3615'
WHEN`City` = 'KABUPATEN OGAN ILIR' then '3616'
WHEN`City` = 'KABUPATEN PENUKAL ABAB LEMATANG ILIR' then '3619'
WHEN`City` = 'KOTA PALEMBANG' then '3691'
WHEN`City` = 'KOTA LUBUKLINGGAU' then '3693'
WHEN`City` = 'KOTA PRABUMULIH' then '3694'
WHEN`City` = 'KOTA PAGAR ALAM' then '3697'
WHEN`City` = 'KABUPATEN MUSI RAWAS UTARA' then '3618'
WHEN`City` = 'KABUPATEN EMPAT LAWANG' then '3617'
WHEN`City` = 'PROVINSI KEPULAUAN BANGKA BELITUNG' then '3700'
WHEN`City` = 'KABUPATEN BANGKA' then '3701'
WHEN`City` = 'KABUPATEN BELITUNG' then '3702'
WHEN`City` = 'KABUPATEN BANGKA BARAT' then '3703'
WHEN`City` = 'KABUPATEN BANGKA SELATAN' then '3704'
WHEN`City` = 'KABUPATEN BANGKA TENGAH' then '3705'
WHEN`City` = 'KABUPATEN BELITUNG TIMUR' then '3706'
WHEN`City` = 'KABUPATEN BANGKA BELITUNG(LBU 2016)/KOTA PANGKAL PINANG' then '3707'
WHEN`City` = 'KOTA PANGKAL PINANG' then '3791'
WHEN`City` = 'PROVINSI KEPULAUAN RIAU' then '3800'
WHEN`City` = 'KABUPATEN KARIMUN' then '3801'
WHEN`City` = 'KABUPATEN LINGGA' then '3802'
WHEN`City` = 'KABUPATEN NATUNA' then '3803'
WHEN`City` = 'KABUPATEN BINTAN' then '3804'
WHEN`City` = 'KABUPATEN KEPULAUAN ANAMBAS' then '3805'
WHEN`City` = 'KOTA TANJUNG PINANG' then '3891'
WHEN`City` = 'KOTA BATAM' then '3892'
WHEN`City` = 'PROVINSI LAMPUNG' then '3900'
WHEN`City` = 'KABUPATEN LAMPUNG SELATAN' then '3901'
WHEN`City` = 'KABUPATEN LAMPUNG TENGAH' then '3902'
WHEN`City` = 'KABUPATEN LAMPUNG UTARA' then '3903'
WHEN`City` = 'KABUPATEN LAMPUNG BARAT' then '3904'
WHEN`City` = 'KABUPATEN TULANG BAWANG' then '3905'
WHEN`City` = 'KABUPATEN TANGGAMUS' then '3906'
WHEN`City` = 'KABUPATEN LAMPUNG TIMUR' then '3907'
WHEN`City` = 'KABUPATEN WAY KANAN' then '3908'
WHEN`City` = 'KABUPATEN PESAWARAN' then '3909'
WHEN`City` = 'KABUPATEN PRINGSEWU' then '3910'
WHEN`City` = 'KABUPATEN TULANG BAWANG BARAT' then '3911'
WHEN`City` = 'KABUPATEN PESISIR BARAT' then '3913'
WHEN`City` = 'KOTA BANDAR LAMPUNG' then '3991'
WHEN`City` = 'KOTA METRO' then '3992'
WHEN`City` = 'KABUPATEN MESUJI' then '3912'
WHEN`City` = 'KABUPATEN BANJAR' then '5101'
WHEN`City` = 'KABUPATEN TANAH LAUT' then '5102'
WHEN`City` = 'KABUPATEN TAPIN' then '5103'
WHEN`City` = 'KABUPATEN HULU SUNGAI SELATAN' then '5104'
WHEN`City` = 'KABUPATEN HULU SUNGAI TENGAH' then '5105'
WHEN`City` = 'KABUPATEN HULU SUNGAI UTARA' then '5106'
WHEN`City` = 'KABUPATEN BARITO KUALA' then '5107'
WHEN`City` = 'KABUPATEN KOTA BARU' then '5108'
WHEN`City` = 'KABUPATEN TABALONG' then '5109'
WHEN`City` = 'KABUPATEN TANAH BUMBU' then '5110'
WHEN`City` = 'KABUPATEN BALANGAN' then '5111'
WHEN`City` = 'KOTA BANJARMASIN' then '5191'
WHEN`City` = 'KOTA BANJARBARU' then '5192'
WHEN`City` = 'PROVINSI KALIMANTAN SELATAN' then '5100'
WHEN`City` = 'PROVINSI KALIMANTAN BARAT' then '5300'
WHEN`City` = 'KABUPATEN PONTIANAK' then '5301'
WHEN`City` = 'KABUPATEN SAMBAS' then '5302'
WHEN`City` = 'KABUPATEN KETAPANG' then '5303'
WHEN`City` = 'KABUPATEN SANGGAU' then '5304'
WHEN`City` = 'KABUPATEN SINTANG' then '5305'
WHEN`City` = 'KABUPATEN KAPUAS HULU' then '5306'
WHEN`City` = 'KABUPATEN BENGKAYANG' then '5307'
WHEN`City` = 'KABUPATEN LANDAK' then '5308'
WHEN`City` = 'KABUPATEN SEKADAU' then '5309'
WHEN`City` = 'KABUPATEN MELAWI' then '5310'
WHEN`City` = 'KABUPATEN KAYONG UTARA' then '5311'
WHEN`City` = 'KABUPATEN KUBU RAYA' then '5312'
WHEN`City` = 'KOTA PONTIANAK' then '5391'
WHEN`City` = 'KOTA SINGKAWANG' then '5392'
WHEN`City` = 'PROVINSI KALIMANTAN TIMUR' then '5400'
WHEN`City` = 'KABUPATEN KUTAI KARTANEGARA' then '5401'
WHEN`City` = 'KABUPATEN BERAU' then '5402'
WHEN`City` = 'KABUPATEN PASER' then '5403'
WHEN`City` = 'KABUPATEN BULUNGAN' then '5404'
WHEN`City` = 'KABUPATEN KUTAI BARAT' then '5405'
WHEN`City` = 'KABUPATEN KUTAI TIMUR' then '5406'
WHEN`City` = 'KABUPATEN NUNUKAN' then '5409'
WHEN`City` = 'KABUPATEN MALINAU' then '5410'
WHEN`City` = 'KABUPATEN PENAJAM PASER UTARA' then '5411'
WHEN`City` = 'KABUPATEN MAHAKAM ULU' then '5413'
WHEN`City` = 'KOTA SAMARINDA' then '5491'
WHEN`City` = 'KOTA BALIKPAPAN' then '5492'
WHEN`City` = 'KOTA TARAKAN' then '5493'
WHEN`City` = 'KOTA BONTANG' then '5494'
WHEN`City` = 'KABUPATEN TANA TIDUNG' then '5412'
WHEN`City` = 'PROVINSI KALIMANTAN TENGAH' then '5800'
WHEN`City` = 'KABUPATEN KAPUAS' then '5801'
WHEN`City` = 'KABUPATEN KOTAWARINGIN BARAT' then '5802'
WHEN`City` = 'KABUPATEN KOTAWARINGIN TIMUR' then '5803'
WHEN`City` = 'KABUPATEN MURUNG RAYA' then '5804'
WHEN`City` = 'KABUPATEN BARITO TIMUR' then '5805'
WHEN`City` = 'KABUPATEN BARITO SELATAN' then '5806'
WHEN`City` = 'KABUPATEN GUNUNG MAS' then '5807'
WHEN`City` = 'KABUPATEN BARITO UTARA' then '5808'
WHEN`City` = 'KABUPATEN PULANG PISAU' then '5809'
WHEN`City` = 'KABUPATEN SERUYAN' then '5810'
WHEN`City` = 'KABUPATEN KATINGAN' then '5811'
WHEN`City` = 'KABUPATEN SUKAMARA' then '5812'
WHEN`City` = 'KABUPATEN LAMANDAU' then '5813'
WHEN`City` = 'KOTA PALANGKARAYA' then '5892'
WHEN`City` = 'PROVINSI SULAWESI TENGAH' then '6000'
WHEN`City` = 'KABUPATEN DONGGALA' then '6001'
WHEN`City` = 'KABUPATEN POSO' then '6002'
WHEN`City` = 'KABUPATEN BANGGAI' then '6003'
WHEN`City` = 'KABUPATEN TOLI-TOLI' then '6004'
WHEN`City` = 'KABUPATEN BANGGAI KEPULAUAN' then '6005'
WHEN`City` = 'KABUPATEN MOROWALI' then '6006'
WHEN`City` = 'KABUPATEN BUOL' then '6007'
WHEN`City` = 'KABUPATEN TOJO UNA-UNA' then '6008'
WHEN`City` = 'KABUPATEN PARIGI MOUTONG' then '6009'
WHEN`City` = 'KABUPATEN SIGI' then '6010'
WHEN`City` = 'KOTA PALU' then '6091'
WHEN`City` = 'KABUPATEN MOROWALI UTARA' then '6012'
WHEN`City` = 'KABUPATEN BANGGAI LAUT' then '6011'
WHEN`City` = 'PROVINSI SULAWESI SELATAN' then '6100'
WHEN`City` = 'KABUPATEN PINRANG' then '6101'
WHEN`City` = 'KABUPATEN GOWA' then '6102'
WHEN`City` = 'KABUPATEN WAJO' then '6103'
WHEN`City` = 'KABUPATEN BONE' then '6105'
WHEN`City` = 'KABUPATEN TANA TORAJA' then '6106'
WHEN`City` = 'KABUPATEN MAROS' then '6107'
WHEN`City` = 'KABUPATEN LUWU' then '6109'
WHEN`City` = 'KABUPATEN SINJAI' then '6110'
WHEN`City` = 'KABUPATEN BULUKUMBA' then '6111'
WHEN`City` = 'KABUPATEN BANTAENG' then '6112'
WHEN`City` = 'KABUPATEN JENEPONTO' then '6113'
WHEN`City` = 'KABUPATEN KEPULAUAN SELAYAR' then '6114'
WHEN`City` = 'KABUPATEN TAKALAR' then '6115'
WHEN`City` = 'KABUPATEN BARRU' then '6116'
WHEN`City` = 'KABUPATEN SIDENRENG RAPPANG' then '6117'
WHEN`City` = 'KABUPATEN PANGKAJENE KEPULAUAN' then '6118'
WHEN`City` = 'KABUPATEN SOPPENG' then '6119'
WHEN`City` = 'KABUPATEN ENREKANG' then '6121'
WHEN`City` = 'KABUPATEN LUWU TIMUR' then '6122'
WHEN`City` = 'KABUPATEN LUWU UTARA' then '6124'
WHEN`City` = 'KABUPATEN TORAJA UTARA' then '6125'
WHEN`City` = 'KOTA MAKASSAR' then '6191'
WHEN`City` = 'KOTA PARE-PARE' then '6192'
WHEN`City` = 'KOTA PALOPO' then '6193'
WHEN`City` = 'PROVINSI SULAWESI UTARA' then '6200'
WHEN`City` = 'KABUPATEN MINAHASA' then '6202'
WHEN`City` = 'KABUPATEN BOLAANG MONGONDOW' then '6203'
WHEN`City` = 'KABUPATEN KEPULAUAN SANGIHE' then '6204'
WHEN`City` = 'KABUPATEN KEPULAUAN TALAUD' then '6205'
WHEN`City` = 'KABUPATEN MINAHASA SELATAN' then '6206'
WHEN`City` = 'KABUPATEN MINAHASA UTARA' then '6207'
WHEN`City` = 'KABUPATEN MINAHASA TENGGARA' then '6209'
WHEN`City` = 'KABUPATEN BOLAANG MONGONDOW UTARA' then '6210'
WHEN`City` = 'KABUPATEN KEPULAUAN SITARO' then '6211'
WHEN`City` = 'KABUPATEN BOLAANG MONGONDOW SELATAN' then '6212'
WHEN`City` = 'KABUPATEN BOLAANG MONGONDOW TIMUR' then '6213'
WHEN`City` = 'KOTA MANADO' then '6291'
WHEN`City` = 'KOTA KOTAMOBAGU' then '6292'
WHEN`City` = 'KOTA BITUNG' then '6293'
WHEN`City` = 'KOTA TOMOHON' then '6294'
WHEN`City` = 'PROVINSI GORONTALO' then '6300'
WHEN`City` = 'KABUPATEN GORONTALO' then '6301'
WHEN`City` = 'KABUPATEN BUALEMO' then '6302'
WHEN`City` = 'KABUPATEN BONEBOLANGO' then '6303'
WHEN`City` = 'KABUPATEN POHUWATO' then '6304'
WHEN`City` = 'KABUPATEN GORONTALO UTARA' then '6305'
WHEN`City` = 'KOTA GORONTALO' then '6391'
WHEN`City` = 'PROVINSI SULAWESI BARAT' then '6400'
WHEN`City` = 'KABUPATEN POLEWALI MANDAR' then '6401'
WHEN`City` = 'KABUPATEN MAJENE' then '6402'
WHEN`City` = 'KABUPATEN MAMASA' then '6403'
WHEN`City` = 'KABUPATEN MAMUJU TENGAH' then '6405'
WHEN`City` = 'KAB MAMUJU' then '6491'
WHEN`City` = 'KABUPATEN MAMUJU UTARA' then '6404'
WHEN`City` = 'KABUPATEN MAMUJU (LBU 2016)' then '6406'
WHEN`City` = 'PROVINSI SULAWESI TENGGARA' then '6900'
WHEN`City` = 'KABUPATEN BUTON' then '6901'
WHEN`City` = 'KABUPATEN MUNA' then '6903'
WHEN`City` = 'KABUPATEN KOLAKA' then '6904'
WHEN`City` = 'KABUPATEN WAKATOBI' then '6905'
WHEN`City` = 'KABUPATEN KONAWE' then '6906'
WHEN`City` = 'KABUPATEN KONAWE SELATAN' then '6907'
WHEN`City` = 'KABUPATEN BOMBANA' then '6908'
WHEN`City` = 'KABUPATEN KOLAKA UTARA' then '6909'
WHEN`City` = 'KABUPATEN BUTON UTARA' then '6910'
WHEN`City` = 'KABUPATEN MUNA BARAT' then '6916'
WHEN`City` = 'KOTA BAU-BAU' then '6990'
WHEN`City` = 'KOTA KENDARI' then '6991'
WHEN`City` = 'KABUPATEN KALOKA TIMUR' then '6912'
WHEN`City` = 'KABUPATEN BUTON TENGAH' then '6915'
WHEN`City` = 'KABUPATEN BUTON SELATAN' then '6914'
WHEN`City` = 'KABUPATEN KONAWE UTARA' then '6911'
WHEN`City` = 'KABUPATEN KONAWE KEPULAUAN' then '6913'
WHEN`City` = 'PROVINSI NUSA TENGGARA BARAT' then '7100'
WHEN`City` = 'KABUPATEN LOMBOK BARAT' then '7101'
WHEN`City` = 'KABUPATEN LOMBOK TENGAH' then '7102'
WHEN`City` = 'KABUPATEN LOMBOK TIMUR' then '7103'
WHEN`City` = 'KABUPATEN SUMBAWA' then '7104'
WHEN`City` = 'KABUPATEN BIMA' then '7105'
WHEN`City` = 'KABUPATEN DOMPU' then '7106'
WHEN`City` = 'KABUPATEN SUMBAWA BARAT' then '7107'
WHEN`City` = 'KABUPATEN LOMBOK UTARA' then '7108'
WHEN`City` = 'KOTA MATARAM' then '7191'
WHEN`City` = 'KOTA BIMA' then '7192'
WHEN`City` = 'PROVINSI BALI' then '7200'
WHEN`City` = 'KABUPATEN BULELENG' then '7201'
WHEN`City` = 'KABUPATEN JEMBRANA' then '7202'
WHEN`City` = 'KABUPATEN TABANAN' then '7203'
WHEN`City` = 'KABUPATEN BADUNG' then '7204'
WHEN`City` = 'KABUPATEN GIANYAR' then '7205'
WHEN`City` = 'KABUPATEN KLUNGKUNG' then '7206'
WHEN`City` = 'KABUPATEN BANGLI' then '7207'
WHEN`City` = 'KABUPATEN KARANGASEM' then '7208'
WHEN`City` = 'KOTA DENPASAR' then '7291'
WHEN`City` = 'PROVINSI NUSA TENGGARA TIMUR' then '7400'
WHEN`City` = 'KABUPATEN KUPANG' then '7401'
WHEN`City` = 'KABUPATEN TIMOR-TENGAH SELATAN' then '7402'
WHEN`City` = 'KABUPATEN TIMOR-TENGAH UTARA' then '7403'
WHEN`City` = 'KABUPATEN BELU' then '7404'
WHEN`City` = 'KABUPATEN ALOR' then '7405'
WHEN`City` = 'KABUPATEN FLORES TIMUR' then '7406'
WHEN`City` = 'KABUPATEN SIKKA' then '7407'
WHEN`City` = 'KABUPATEN ENDE' then '7408'
WHEN`City` = 'KABUPATEN NGADA' then '7409'
WHEN`City` = 'KABUPATEN MANGGARAI' then '7410'
WHEN`City` = 'KABUPATEN SUMBA TIMUR' then '7411'
WHEN`City` = 'KABUPATEN SUMBA BARAT' then '7412'
WHEN`City` = 'KABUPATEN LEMBATA' then '7413'
WHEN`City` = 'KABUPATEN ROTE NDAO' then '7414'
WHEN`City` = 'KABUPATEN MANGGARAI BARAT' then '7415'
WHEN`City` = 'KABUPATEN SUMBA TENGAH' then '7416'
WHEN`City` = 'KABUPATEN SUMBA BARAT DAYA' then '7417'
WHEN`City` = 'KABUPATEN MANGGARAI TIMUR' then '7418'
WHEN`City` = 'KABUPATEN NAGEKEO' then '7419'
WHEN`City` = 'KABUPATEN MALAKA' then '7421'
WHEN`City` = 'KOTA KUPANG' then '7491'
WHEN`City` = 'KABUPATEN SABU RAIJUA' then '7420'
WHEN`City` = 'PROVINSI MALUKU' then '8100'
WHEN`City` = 'KABUPATEN MALUKU TENGAH' then '8101'
WHEN`City` = 'KABUPATEN MALUKU TENGGARA' then '8102'
WHEN`City` = 'KABUPATEN MALUKU TENGGARA BARAT' then '8103'
WHEN`City` = 'KABUPATEN BURU' then '8104'
WHEN`City` = 'KABUPATEN SERAM BAGIAN BARAT' then '8105'
WHEN`City` = 'KABUPATEN SERAM BAGIAN TIMUR' then '8106'
WHEN`City` = 'KABUPATEN KEPULAUAN ARU' then '8107'
WHEN`City` = 'KABUPATEN MALUKU BARAT DAYA' then '8108'
WHEN`City` = 'KABUPATEN BURU SELATAN' then '8109'
WHEN`City` = 'KOTA AMBON' then '8191'
WHEN`City` = 'KOTA TUAL' then '8192'
WHEN`City` = 'PROVINSI PAPUA' then '8200'
WHEN`City` = 'KABUPATEN JAYAPURA' then '8201'
WHEN`City` = 'KABUPATEN BIAK NUMFOR' then '8202'
WHEN`City` = 'KABUPATEN KEPULAUAN YAPEN-WAROPEN' then '8210'
WHEN`City` = 'KABUPATEN MERAUKE' then '8211'
WHEN`City` = 'KABUPATEN PANIAI' then '8212'
WHEN`City` = 'KABUPATEN JAYAWIJAYA' then '8213'
WHEN`City` = 'KABUPATEN NABIRE' then '8214'
WHEN`City` = 'KABUPATEN MIMIKA' then '8215'
WHEN`City` = 'KABUPATEN PUNCAK JAYA' then '8216'
WHEN`City` = 'KABUPATEN SARMI' then '8217'
WHEN`City` = 'KABUPATEN KEEROM' then '8218'
WHEN`City` = 'KABUPATEN PEGUNUNGAN BINTANG' then '8221'
WHEN`City` = 'KABUPATEN YAHUKIMO' then '8222'
WHEN`City` = 'KABUPATEN TOLIKARA' then '8223'
WHEN`City` = 'KABUPATEN WAROPEN' then '8224'
WHEN`City` = 'KABUPATEN BOVEN DIGOEL' then '8226'
WHEN`City` = 'KABUPATEN MAPPI' then '8227'
WHEN`City` = 'KABUPATEN ASMAT' then '8228'
WHEN`City` = 'KABUPATEN SUPIORI' then '8231'
WHEN`City` = 'KABUPATEN MAMBERAMO RAYA' then '8232'
WHEN`City` = 'KABUPATEN DOGIYAI' then '8233'
WHEN`City` = 'KABUPATEN LANNY JAYA' then '8234'
WHEN`City` = 'KABUPATEN MAMBERAMO TENGAH' then '8235'
WHEN`City` = 'KABUPATEN NDUGA' then '8236'
WHEN`City` = 'KABUPATEN YALIMO' then '8237'
WHEN`City` = 'KABUPATEN PUNCAK' then '8238'
WHEN`City` = 'KABUPATEN DEIYA' then '8240'
WHEN`City` = 'KOTA JAYAPURA' then '8291'
WHEN`City` = 'KABUPATEN INTAN JAYA' then '8239'
WHEN`City` = 'PROVINSI MALUKU UTARA' then '8300'
WHEN`City` = 'KABUPATEN HALMAHERA TENGAH' then '8302'
WHEN`City` = 'KABUPATEN HALMAHERA UTARA' then '8303'
WHEN`City` = 'KABUPATEN HALMAHERA TIMUR' then '8304'
WHEN`City` = 'KABUPATEN HALMAHERA BARAT' then '8305'
WHEN`City` = 'KABUPATEN HALMAHERA SELATAN' then '8306'
WHEN`City` = 'KABUPATEN KEPULAUAN SULA' then '8307'
WHEN`City` = 'KABUPATEN PULAU MOROTAI' then '8308'
WHEN`City` = 'KOTA TERNATE' then '8390'
WHEN`City` = 'KOTA TIDORE KEPULAUAN' then '8391'
WHEN`City` = 'KABUPATEN PULAU TALIABU' then '8309'
WHEN`City` = 'PROVINSI PAPUA BARAT' then '8400'
WHEN`City` = 'KABUPATEN SORONG' then '8401'
WHEN`City` = 'KABUPATEN FAK-FAK' then '8402'
WHEN`City` = 'KABUPATEN MANOKWARI' then '8403'
WHEN`City` = 'KABUPATEN SORONG SELATAN' then '8404'
WHEN`City` = 'KABUPATEN RAJA AMPAT' then '8405'
WHEN`City` = 'KABUPATEN KAIMANA' then '8406'
WHEN`City` = 'KABUPATEN TELUK BINTUNI' then '8407'
WHEN`City` = 'KABUPATEN TELUK WONDAMA' then '8408'
WHEN`City` = 'KABUPATEN TEMBRAUW' then '8409'
WHEN`City` = 'KABUPATEN MAYBRAT' then '8410'
WHEN`City` = 'KOTA SORONG' then '8491'
WHEN`City` = 'KABUPATEN PEGUNUNGAN ARFAK' then '8411'
WHEN`City` = 'KABUPATEN MANOKWARI SELATAN' then '8412'
WHEN`City` = 'KABUPATEN BANYU ASIN' then '3607'
WHEN`City` = 'JAKARTA PUSAT' then '0391'
WHEN`City` = 'DI LUAR INDONESIA' then '9999'

           ELSE '0000'
end as `Kode_Kabupaten_atau_Kota_Lokasi_Proyek_atau_Penggunaan_Kredit_atau_Pembiayaan`,
`Nilai_Proyek`,
`Kode_Valuta`,
`Suku_Bunga_atau_Imbalan`,
`Jenis_Suku_Bunga_atau_Imbalan`,
`Kredit_atau_Pembiayaan_Program_Pemerintah`,
`Plafon_Awal`,
`Plafon`,
`Realisasi_atau_Pencairan_Bulan_Berjalan`,
`Denda`,
`Baki_Debet`,
`Nilai_dalam_Mata_Uang_Asal`,
`Kode_Kualitas_Kredit_atau_Pembiayaan`,
`Tanggal_Macet`,
`Kode_Sebab_Macet`,
`Tunggakan_Pokok`,
`Tunggakan_Bunga_atau_Imbalan`,
`Jumlah_Hari_Tunggakan`,
`Frekuensi_Tunggakan`,
`Frekuensi_Restrukturisasi`,
`Tanggal_Restrukturisasi_Awal`,
`Tanggal_Restrukturisasi_Akhir`,
`Kode_Cara_Restrukturisasi`,
`Kode_Kondisi`,
FORMAT_DATE('%Y%m%d',date(`Tanggal_Kondisi`)) `Tanggal_Kondisi`,
`Keterangan`,
`Kode_Kantor_Cabang`,
`Operasi_Data`,
`Status_delete`,
`Create_Date`,
`Update_Date`

from `summary`


In [5]:
print(F01new.shape)
a=[]
for i in F01new.columns:
    a.append(i.replace('_',' '))
print(a)
F01new.columns = a
F01new.head(3)

(348107, 53)
['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data', 'Kode Jenis Fasilitas', 'Nomor Rekening Fasilitas', 'Nomor Rekening Lama Fasilitas', 'Nomor CIF Debitur', 'Kode Sifat Kredit atau Pembiayaan', 'Kode Jenis Kredit atau Pembiayaan', 'Kode Akad Kredit atau Akad Pembiayaan', 'Nomor Akad Awal', 'Tanggal Akad Awal', 'Nomor Akad Akhir', 'Tanggal Akad Akhir', 'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan', 'Tanggal Awal Kredit atau Pembiayaan', 'Tanggal Mulai', 'Tanggal Jatuh Tempo', 'Kode Kategori Debitur', 'Kode Jenis Penggunaan', 'Kode Orientasi Penggunaan', 'Kode Sektor Ekonomi', 'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan', 'Nilai Proyek', 'Kode Valuta', 'Suku Bunga atau Imbalan', 'Jenis Suku Bunga atau Imbalan', 'Kredit atau Pembiayaan Program Pemerintah', 'Plafon Awal', 'Plafon', 'Realisasi atau Pencairan Bulan Berjalan', 'Denda', 'Baki Debet', 'Nilai dalam Mata Uang Asal', 'Kode Kualitas Kredit atau Pembiayaan', 'Tangga

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Kode Jenis Fasilitas,Nomor Rekening Fasilitas,Nomor Rekening Lama Fasilitas,Nomor CIF Debitur,Kode Sifat Kredit atau Pembiayaan,Kode Jenis Kredit atau Pembiayaan,Kode Akad Kredit atau Akad Pembiayaan,...,Tanggal Restrukturisasi Akhir,Kode Cara Restrukturisasi,Kode Kondisi,Tanggal Kondisi,Keterangan,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date
0,0301,900013,201911,F01,ATH-JNLG2R36,None,3915917686490,9,05,00,...,,,00,None,None,000,C,T,20190919010159,20191105180050
1,0301,900013,201911,F01,ATH-73ENPD7J,None,3915909418575,9,05,00,...,,,00,None,None,000,C,T,20191018044003,20191105174849
2,0301,900013,201911,F01,ATH-DX8E8D6T,None,3915971232859,9,05,00,...,,,00,None,None,000,C,T,20190116061231,20191105182501


In [6]:
#creditdp = rf.get_fresh_query_result('https://reports.indodana.com/', 3378, 'XC1J54mmAqO59hH0TB8cOFMNJ6gVyeJaj7EmMV8r', {'p_param': 3378})
#creditprod = rf.get_fresh_query_result('https://reports.indodana.com/', 3379, 'XC1J54mmAqO59hH0TB8cOFMNJ6gVyeJaj7EmMV8r', {'p_param': 3379})

In [7]:
#creditdp['source']= 'Data Platform'
#creditprod['source'] = 'Production'
#creditdp['limit'] = creditdp['limit'].astype(float)
#creditdp['limit_balance'] = creditdp['limit_balance'].astype(float)
#creditprod['limit'] = creditprod['limit'].astype(float)
#creditprod['limit_balance'] = creditprod['limit_balance'].astype(float)

In [8]:
#creditprod.shape

In [9]:
#credit=pd.concat([creditdp,creditprod]).drop_duplicates(subset=['application_order_id','borrower_id','limit','limit_balance'],keep=False)
#credit.shape

Data Preparation Process for D01 and F01 KBIJ =
1. Import previous data 
2. Import full data dump from query 
3. Find Delta between new data dump and previous data 
4. Export delta to csv and send to KBIJ 
5. Export full data dump to csv to be used in the next iteration

Scrapped code for first time only

In [10]:
#F01 = pd.read_csv('SEGMEN_F01_–_FASILITAS_KREDIT_2019_09_03.csv',dtype=str)
#D01 = pd.read_csv('SEGMEN_D01_–_DEBITUR_INDIVIDUAL_PERORANGAN_2019_09_03.csv',dtype=str)
#F01.to_csv(path_or_buf='F01.csv', index =0)
#D01.to_csv(path_or_buf='D01.csv', index =0)

Import Previous Data

Revise F01 08 09

In [11]:
# finalrev = F0108rev.loc[F0108rev['Kode Kualitas Kredit atau Pembiayaan'].isna()]
# finalrev['Kode Kualitas Kredit atau Pembiayaan'] = finalrev['Kode Kualitas Kredit atau Pembiayaan'].fillna(1)

In [12]:
# finalrev.to_csv(path_or_buf='F01-201908revfinal0.txt', sep='|',index =0)

In [13]:
F0109rev = pd.read_csv('F01-201909.txt',dtype=str,sep='|')
F0109 = pd.read_csv('F01-201909rev.txt',dtype=str,sep='|')

In [1]:
F0108rev = pd.read_csv('F01-201908.txt',dtype=str,sep='|')
F0109rev = pd.read_csv('F01-201909.txt',dtype=str,sep='|')
F0108rev = F0108rev.replace('\n','', regex=True)
F0109rev = F0109rev.replace('\n','', regex=True)

In [7]:
# F0108rev['Kode Kualitas Kredit atau Pembiayaan'] = F0108rev['Kode Kualitas Kredit atau Pembiayaan'].astype(str)     
F0108rev['Kode Sebab Macet'] = F0108rev['Kode Sebab Macet'].str.replace('.0', '')
F0108rev['Kode Kualitas Kredit atau Pembiayaan'] = F0108rev['Kode Kualitas Kredit atau Pembiayaan'].fillna(1)
F0108rev['Kode Kualitas Kredit atau Pembiayaan'] = F0108rev['Kode Kualitas Kredit atau Pembiayaan'].str.replace('.0', '')
F0108rev['Tanggal Akad Awal']=F0108rev['Tanggal Awal Kredit atau Pembiayaan']

In [8]:
# F0109rev['Kode Sebab Macet'] = F0109rev['Kode Sebab Macet'].astype(str)          
F0109rev['Kode Sebab Macet'] = F0109rev['Kode Sebab Macet'].str.replace('.0', '')
# F0109rev['Kode Kualitas Kredit atau Pembiayaan'] = F0109rev['Kode Kualitas Kredit atau Pembiayaan'].astype(str)          
F0109rev['Kode Kualitas Kredit atau Pembiayaan'] = F0109rev['Kode Kualitas Kredit atau Pembiayaan'].fillna(1)
F0109rev['Kode Kualitas Kredit atau Pembiayaan'] = F0109rev['Kode Kualitas Kredit atau Pembiayaan'].str.replace('.0', '')
# F0109rev['Kode Sebab Macet'] = F0109rev['Kode Sebab Macet'].str.replace(' ', '')
F0109rev['Tanggal Akad Awal']=F0109rev['Tanggal Awal Kredit atau Pembiayaan']

In [10]:
F0109rev['Kode Kualitas Kredit atau Pembiayaan'] = F0109rev['Kode Kualitas Kredit atau Pembiayaan'].fillna(1)

In [11]:
F0109rev.loc[F0109rev['Kode Kualitas Kredit atau Pembiayaan'].isna()].shape

(0, 53)

In [118]:
F0108rev.to_csv(path_or_buf='F01-201908rev.txt', sep='|',index =0)

In [12]:
F0109rev.to_csv(path_or_buf='F01-201909rev.txt', sep='|',index =0)

In [9]:
F01 = pd.read_csv('F01.csv',dtype=str)
D01 = pd.read_csv('D01.csv',dtype=str)

For Month 10 Upload, wrong output previous data

In [146]:
F01 = pd.read_csv('F01201909correct.csv',dtype=str)
# D01 = pd.read_csv('F01201909.csv',dtype=str)
D01 = pd.read_csv('D01201909.csv',dtype=str)

In [147]:
F01 = F01.replace('\n','', regex=True)
D01 = D01.replace('\n','', regex=True)

In [198]:
#Create F01201909correct because the old one is broken
# F01 = pd.read_csv('F01-201909rev.txt',dtype=str,sep='|')
# F01 = F01.replace('\n','', regex=True)
# print(F01.shape)
# tes = pd.read_csv('F01.csv',dtype=str)
# tes['Kode Kualitas Kredit atau Pembiayaan'] = tes['Kode Kualitas Kredit atau Pembiayaan ']
# tes['Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan'] = tes['Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit a']
# tes = tes.drop(columns=['Kode Kualitas Kredit atau Pembiayaan ','Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit a'])
# print(tes.shape)
# F01201908 = pd.read_csv('F01-201908rev.txt',dtype=str,sep='|')
# print(F01201908.shape)
# F01old = pd.concat([F01,F01201908,tes])
# print('concatenate result of 08 old, 08 revision, 09 sent',F01old.shape)
# F01old['date'] = pd.to_datetime(F01old['Update Date'])
# F01old = F01old.sort_values(by=['Nomor Rekening Fasilitas','date'])
# F01old = F01old.drop_duplicates(subset=['Nomor Rekening Fasilitas'],keep='last')
# F01old = F01old.drop(columns=['date'])
# print(F01old.shape)
# F01old.to_csv(path_or_buf='F01201909correct.csv', index=0)

(360334, 53)

In [24]:
D01.shape

(107648, 35)

In [25]:
D01new.shape

(127309, 35)

In [26]:
D01new.columns

Index(['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Nomor CIF Lama Debitur', 'Nomor CIF Debitur', 'Jenis Identitas',
       'Nomor Identitas', 'Nama Sesuai Identitas', 'Nama Lengkap',
       'Kode Status Pendidikan atau Gelar Debitur', 'Jenis Kelamin',
       'Tempat Lahir', 'Tanggal Lahir', 'Nama Gadis Ibu Kandung', 'NPWP',
       'Alamat', 'Kelurahan', 'Kecamatan', 'Kode Kabupaten atau Kota',
       'Kode Pos', 'Nomor Telepon', 'Nomor Telepon Seluler', 'Alamat e-mail',
       'Kode Negara Domisili', 'Kode Pekerjaan', 'Tempat Bekerja',
       'Kode Bidang Usaha Tempat Bekerja', 'Alamat Tempat Bekerja',
       'Kode Golongan Debitur', 'Status Perkawinan Debitur',
       'Kode Kantor Cabang', 'Operasi Data', 'Status delete', 'Create Date',
       'Update Date'],
      dtype='object')

Get new F01 and D01 from redash

In [127]:
D01new = rf.get_fresh_query_result('https://reports.indodana.com/', 2302, 'XC1J54mmAqO59hH0TB8cOFMNJ6gVyeJaj7EmMV8r', {'p_param': 2302})

In [ ]:
D01new = D01new[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Nomor CIF Lama Debitur', 'Nomor CIF Debitur', 'Jenis Identitas',
       'Nomor Identitas', 'Nama Sesuai Identitas', 'Nama Lengkap',
       'Kode Status Pendidikan atau Gelar Debitur', 'Jenis Kelamin',
       'Tempat Lahir', 'Tanggal Lahir', 'Nama Gadis Ibu Kandung', 'NPWP',
       'Alamat', 'Kelurahan', 'Kecamatan', 'Kode Kabupaten atau Kota',
       'Kode Pos', 'Nomor Telepon', 'Nomor Telepon Seluler', 'Alamat e-mail',
       'Kode Negara Domisili', 'Kode Pekerjaan', 'Tempat Bekerja',
       'Kode Bidang Usaha Tempat Bekerja', 'Alamat Tempat Bekerja',
       'Kode Golongan Debitur', 'Status Perkawinan Debitur',
       'Kode Kantor Cabang', 'Operasi Data', 'Status delete', 'Create Date',
       'Update Date']]

In [ ]:
print(F01new.shape,D01new.shape)

In [29]:
# D01new = pd.read_csv('D01201910.csv',dtype=str)
# D01new.shape

(124209, 35)

In [ ]:
F01new = F01new.replace('\n','', regex=True)
D01new = D01new.replace('\n','', regex=True)

In [31]:
print('D01:',D01.shape, 'F01:',F01.shape)
print('D01new:',D01new.shape, 'F01new:',F01new.shape)
print('D01delta:',D01new.shape[0]-D01.shape[0],'F01delta:',F01new.shape[0]-F01.shape[0])

D01: (107648, 35) F01: (301352, 53)
D01new: (127309, 35) F01new: (348107, 54)
D01delta: 19661 F01delta: 46755


# Cleansing F01 Identify Create Population

In [34]:
F01.shape

(301352, 53)

In [35]:
F01new.shape

(348107, 54)

In [36]:
ff3 = pd.concat([F01,F01new]).sort_values(by=['Update Date'])

In [37]:
ff3 = pd.concat([F01,F01new]).drop_duplicates(subset=['Nomor Rekening Fasilitas'],keep=False)
ff3.shape

(47092, 54)

In [38]:
datem = datetime.today()- timedelta(days=30)
datem = datem.strftime("%Y%m")

In [39]:
ff3['Tahun Bulan Data'] = datem
ff3['date'] = pd.to_datetime(ff3['Update Date'])

In [40]:
ff3['date'].dt.month.value_counts()

11    41124
10     5843
9        59
5        12
6        10
4         9
2         9
8         6
7         6
3         6
1         5
12        3
Name: date, dtype: int64

In [41]:
ff3['sign'] = 1

In [42]:
ff3['Tahun Bulan Data'].value_counts()

201910    47092
Name: Tahun Bulan Data, dtype: int64

In [43]:
ff3['date'].dt.month.value_counts()

11    41124
10     5843
9        59
5        12
6        10
4         9
2         9
8         6
7         6
3         6
1         5
12        3
Name: date, dtype: int64

In [37]:
# ff39 = ff3.loc[ff3['date']<'2019-09-01']
# ff30 = ff3.loc[ff3['date']>='2019-09-01']
# print(ff39.shape,ff30.shape)
# print(ff39.shape[0]+ff30.shape[0])
# ff39['Tahun Bulan Data'] = '201908'
# ff30['Tahun Bulan Data'] = '201909'
# print(ff39['Tahun Bulan Data'].value_counts())
# ff39=ff39[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
#        'Kode Jenis Fasilitas', 'Nomor Rekening Fasilitas',
#        'Nomor Rekening Lama Fasilitas', 'Nomor CIF Debitur',
#        'Kode Sifat Kredit atau Pembiayaan',
#        'Kode Jenis Kredit atau Pembiayaan',
#        'Kode Akad Kredit atau Akad Pembiayaan', 'Nomor Akad Awal',
#        'Tanggal Akad Awal', 'Nomor Akad Akhir', 'Tanggal Akad Akhir',
#        'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan',
#        'Tanggal Awal Kredit atau Pembiayaan', 'Tanggal Mulai',
#        'Tanggal Jatuh Tempo', 'Kode Kategori Debitur', 'Kode Jenis Penggunaan',
#        'Kode Orientasi Penggunaan', 'Kode Sektor Ekonomi',
#        'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan',
#        'Nilai Proyek', 'Kode Valuta', 'Suku Bunga atau Imbalan',
#        'Jenis Suku Bunga atau Imbalan',
#        'Kredit atau Pembiayaan Program Pemerintah', 'Plafon Awal', 'Plafon',
#        'Realisasi atau Pencairan Bulan Berjalan', 'Denda', 'Baki Debet',
#        'Nilai dalam Mata Uang Asal', 'Kode Kualitas Kredit atau Pembiayaan',
#        'Tanggal Macet', 'Kode Sebab Macet', 'Tunggakan Pokok',
#        'Tunggakan Bunga atau Imbalan', 'Jumlah Hari Tunggakan',
#        'Frekuensi Tunggakan', 'Frekuensi Restrukturisasi',
#        'Tanggal Restrukturisasi Awal', 'Tanggal Restrukturisasi Akhir',
#        'Kode Cara Restrukturisasi', 'Kode Kondisi', 'Tanggal Kondisi',
#        'Keterangan', 'Kode Kantor Cabang', 'Operasi Data', 'Status delete',
#        'Create Date', 'Update Date']]
# print(ff30['Tahun Bulan Data'].value_counts())
# ff30=ff30[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
#        'Kode Jenis Fasilitas', 'Nomor Rekening Fasilitas',
#        'Nomor Rekening Lama Fasilitas', 'Nomor CIF Debitur',
#        'Kode Sifat Kredit atau Pembiayaan',
#        'Kode Jenis Kredit atau Pembiayaan',
#        'Kode Akad Kredit atau Akad Pembiayaan', 'Nomor Akad Awal',
#        'Tanggal Akad Awal', 'Nomor Akad Akhir', 'Tanggal Akad Akhir',
#        'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan',
#        'Tanggal Awal Kredit atau Pembiayaan', 'Tanggal Mulai',
#        'Tanggal Jatuh Tempo', 'Kode Kategori Debitur', 'Kode Jenis Penggunaan',
#        'Kode Orientasi Penggunaan', 'Kode Sektor Ekonomi',
#        'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan',
#        'Nilai Proyek', 'Kode Valuta', 'Suku Bunga atau Imbalan',
#        'Jenis Suku Bunga atau Imbalan',
#        'Kredit atau Pembiayaan Program Pemerintah', 'Plafon Awal', 'Plafon',
#        'Realisasi atau Pencairan Bulan Berjalan', 'Denda', 'Baki Debet',
#        'Nilai dalam Mata Uang Asal', 'Kode Kualitas Kredit atau Pembiayaan',
#        'Tanggal Macet', 'Kode Sebab Macet', 'Tunggakan Pokok',
#        'Tunggakan Bunga atau Imbalan', 'Jumlah Hari Tunggakan',
#        'Frekuensi Tunggakan', 'Frekuensi Restrukturisasi',
#        'Tanggal Restrukturisasi Awal', 'Tanggal Restrukturisasi Akhir',
#        'Kode Cara Restrukturisasi', 'Kode Kondisi', 'Tanggal Kondisi',
#        'Keterangan', 'Kode Kantor Cabang', 'Operasi Data', 'Status delete',
#        'Create Date', 'Update Date']]

(197804, 55) (48665, 55)
246469
201908    197804
Name: Tahun Bulan Data, dtype: int64
201909    48665
Name: Tahun Bulan Data, dtype: int64


In [28]:
#ff39.to_csv(path_or_buf='F01-201908.txt', sep='|',index =0)

# F01 identify Update population

In [ ]:
F01['date']= pd.to_datetime(F01['Update Date'])
F01new['date']= pd.to_datetime(F01new['Update Date'])
print(F01['date'].max())
print(F01new['date'].max())

In [ ]:
fu3 = pd.concat([F01,F01new])
fu3['date'] = pd.to_datetime(fu3['Update Date'])
fu3 = fu3.sort_values(by=['Nomor Rekening Fasilitas','date'])

In [ ]:
fu3 = fu3.drop_duplicates(subset=['Nomor Rekening Fasilitas'],keep='last')
fu3.shape

## get additional data for missing data request from KBIJ

In [76]:
# datahilang = pd.read_csv('Indodana-F01-201909-2-missingdata.csv',dtype=str)
# datahilang = datahilang[['Nomor Rekening Fasilitas']].merge(fu3,how='inner',on='Nomor Rekening Fasilitas')
# datahilang['Operasi Data']='U'
# datahilang=datahilang[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
#        'Kode Jenis Fasilitas', 'Nomor Rekening Fasilitas',
#        'Nomor Rekening Lama Fasilitas', 'Nomor CIF Debitur',
#        'Kode Sifat Kredit atau Pembiayaan',
#        'Kode Jenis Kredit atau Pembiayaan',
#        'Kode Akad Kredit atau Akad Pembiayaan', 'Nomor Akad Awal',
#        'Tanggal Akad Awal', 'Nomor Akad Akhir', 'Tanggal Akad Akhir',
#        'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan',
#        'Tanggal Awal Kredit atau Pembiayaan', 'Tanggal Mulai',
#        'Tanggal Jatuh Tempo', 'Kode Kategori Debitur', 'Kode Jenis Penggunaan',
#        'Kode Orientasi Penggunaan', 'Kode Sektor Ekonomi',
#        'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan',
#        'Nilai Proyek', 'Kode Valuta', 'Suku Bunga atau Imbalan',
#        'Jenis Suku Bunga atau Imbalan',
#        'Kredit atau Pembiayaan Program Pemerintah', 'Plafon Awal', 'Plafon',
#        'Realisasi atau Pencairan Bulan Berjalan', 'Denda', 'Baki Debet',
#        'Nilai dalam Mata Uang Asal', 'Kode Kualitas Kredit atau Pembiayaan',
#        'Tanggal Macet', 'Kode Sebab Macet', 'Tunggakan Pokok',
#        'Tunggakan Bunga atau Imbalan', 'Jumlah Hari Tunggakan',
#        'Frekuensi Tunggakan', 'Frekuensi Restrukturisasi',
#        'Tanggal Restrukturisasi Awal', 'Tanggal Restrukturisasi Akhir',
#        'Kode Cara Restrukturisasi', 'Kode Kondisi', 'Tanggal Kondisi',
#        'Keterangan', 'Kode Kantor Cabang', 'Operasi Data', 'Status delete',
#        'Create Date', 'Update Date']]
# datahilang.to_csv(path_or_buf='F01-201910rev1additional.txt', sep='|',index =0)

Filter update by previous max date

In [239]:
fu3 = fu3.loc[fu3['date']>F01['date'].max()]
fu3.shape

(111861, 54)

In [48]:
filterfu3 = fu3.loc[fu3['date']>F01['date'].max()]
filterfu3.shape

(111861, 54)

In [240]:
fu3 = fu3.merge(ff3[['Nomor Rekening Fasilitas','sign']], left_on='Nomor Rekening Fasilitas', right_on='Nomor Rekening Fasilitas', how='left')

Filter create

In [241]:
fu3 = fu3.loc[~(fu3['sign']==1)]
fu3['Tahun Bulan Data'] = datem
fu3['Operasi Data'] = 'U'
fu3.shape

(65406, 55)

In [243]:
# fu3 = pd.concat([fu3,ff30])
fu3 = pd.concat([fu3,ff3])
fu3 = fu3.drop(columns=['date','sign'])
fu3.shape

(112498, 53)

# Cleansing D01 
- Finding the delta between the new D01 and old D01. The resulting dataframe will be the new debtors created after the old D01.

## Identify Create Population D01

In [149]:
df3 = pd.concat([D01,D01new]).drop_duplicates(subset=['Nomor CIF Debitur'],keep=False)
df3.shape

(21216, 35)

In [150]:
df3['date'] = pd.to_datetime(df3['Create Date'])
df3['signc'] = 1

In [31]:
# File to Send to KBIJ D01
#df3.to_csv(path_or_buf='D01-201910.txt', sep='|',index =0)

## Identify Not Update Population D01 (All Active)

Identify Active by Merging Complete D01 data with reported F01 data

In [151]:
factive = fu3[['Nomor CIF Debitur']].drop_duplicates()
factive['sign'] = 1

In [152]:
# merge to get active D01
dn3=D01new.merge(factive,on='Nomor CIF Debitur', how='inner')
dn3['Operasi Data'] = 'N'
print(dn3.shape)
# remove cancelled D01
dn3=dn3.merge(df3[['Nomor CIF Debitur','signc']],on='Nomor CIF Debitur', how='left')
dn3=dn3.loc[~(dn3['signc']==1)]
print(dn3.shape)

(89819, 36)
(75042, 37)


Combine Create df3 and Not Active dn3

In [153]:
dn3 = pd.concat([dn3,df3])
dn3['Tahun Bulan Data'] = datem
dn3 = dn3.drop(columns=['date','sign','signc'])
dn3.shape

(96258, 35)

Remove F01 without D01 

In [101]:
print('before ',fu3.shape)
fu3 = fu3.merge(dn3[['Nomor CIF Debitur']],how='inner',on='Nomor CIF Debitur')
print('after ',fu3.shape)

before  (112498, 53)
after  (112319, 53)


# Final File to Send to KBIJ D01 and F01

In [27]:
# Fix F01-201910rev.txt
# fu3 = pd.read_csv('F01-201910.txt', dtype=str, sep='|')
# fu3=fu3[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
#        'Kode Jenis Fasilitas', 'Nomor Rekening Fasilitas',
#        'Nomor Rekening Lama Fasilitas', 'Nomor CIF Debitur',
#        'Kode Sifat Kredit atau Pembiayaan',
#        'Kode Jenis Kredit atau Pembiayaan',
#        'Kode Akad Kredit atau Akad Pembiayaan', 'Nomor Akad Awal',
#        'Tanggal Akad Awal', 'Nomor Akad Akhir', 'Tanggal Akad Akhir',
#        'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan',
#        'Tanggal Awal Kredit atau Pembiayaan', 'Tanggal Mulai',
#        'Tanggal Jatuh Tempo', 'Kode Kategori Debitur', 'Kode Jenis Penggunaan',
#        'Kode Orientasi Penggunaan', 'Kode Sektor Ekonomi',
#        'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan',
#        'Nilai Proyek', 'Kode Valuta', 'Suku Bunga atau Imbalan',
#        'Jenis Suku Bunga atau Imbalan',
#        'Kredit atau Pembiayaan Program Pemerintah', 'Plafon Awal', 'Plafon',
#        'Realisasi atau Pencairan Bulan Berjalan', 'Denda', 'Baki Debet',
#        'Nilai dalam Mata Uang Asal', 'Kode Kualitas Kredit atau Pembiayaan',
#        'Tanggal Macet', 'Kode Sebab Macet', 'Tunggakan Pokok',
#        'Tunggakan Bunga atau Imbalan', 'Jumlah Hari Tunggakan',
#        'Frekuensi Tunggakan', 'Frekuensi Restrukturisasi',
#        'Tanggal Restrukturisasi Awal', 'Tanggal Restrukturisasi Akhir',
#        'Kode Cara Restrukturisasi', 'Kode Kondisi', 'Tanggal Kondisi',
#        'Keterangan', 'Kode Kantor Cabang', 'Operasi Data', 'Status delete',
#        'Create Date', 'Update Date']]
# print(fu3.shape)
# fu3.to_csv(path_or_buf='F01-201910rev.txt', sep='|',index =0)

In [257]:
#debugging
#F01.to_csv('datacreate.csv') 
#fu3.to_csv('dataupdate.csv')

Null check for CIF and NIK

In [156]:
dn3.loc[(dn3['Nomor CIF Debitur'].isna())|(dn3['Nama Lengkap'].isna())|(dn3['Tanggal Lahir'].isna())|(dn3['Nomor Identitas'].isna())]

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Nomor CIF Lama Debitur,Nomor CIF Debitur,Jenis Identitas,Nomor Identitas,Nama Sesuai Identitas,Nama Lengkap,Kode Status Pendidikan atau Gelar Debitur,...,Tempat Bekerja,Kode Bidang Usaha Tempat Bekerja,Alamat Tempat Bekerja,Kode Golongan Debitur,Status Perkawinan Debitur,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date


In [157]:
fu3.loc[fu3['Nomor CIF Debitur'].isna()]

,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Kode Jenis Fasilitas,Nomor Rekening Fasilitas,Nomor Rekening Lama Fasilitas,Nomor CIF Debitur,Kode Sifat Kredit atau Pembiayaan,Kode Jenis Kredit atau Pembiayaan,Kode Akad Kredit atau Akad Pembiayaan,...,Tanggal Restrukturisasi Akhir,Kode Cara Restrukturisasi,Kode Kondisi,Tanggal Kondisi,Keterangan,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date


Export Latest Data

In [255]:
timestr = time.strftime("%Y%m%d-%H%M%S")
F01name = 'F01'+'-'+datem+'.txt'
D01name = 'D01'+'-'+datem+'.txt'
print(F01name,D01name)

F01-201910.txt D01-201910.txt


In [257]:
D01name

'D01-201910.txt'

In [155]:
fu3=fu3[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Kode Jenis Fasilitas', 'Nomor Rekening Fasilitas',
       'Nomor Rekening Lama Fasilitas', 'Nomor CIF Debitur',
       'Kode Sifat Kredit atau Pembiayaan',
       'Kode Jenis Kredit atau Pembiayaan',
       'Kode Akad Kredit atau Akad Pembiayaan', 'Nomor Akad Awal',
       'Tanggal Akad Awal', 'Nomor Akad Akhir', 'Tanggal Akad Akhir',
       'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan',
       'Tanggal Awal Kredit atau Pembiayaan', 'Tanggal Mulai',
       'Tanggal Jatuh Tempo', 'Kode Kategori Debitur', 'Kode Jenis Penggunaan',
       'Kode Orientasi Penggunaan', 'Kode Sektor Ekonomi',
       'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan',
       'Nilai Proyek', 'Kode Valuta', 'Suku Bunga atau Imbalan',
       'Jenis Suku Bunga atau Imbalan',
       'Kredit atau Pembiayaan Program Pemerintah', 'Plafon Awal', 'Plafon',
       'Realisasi atau Pencairan Bulan Berjalan', 'Denda', 'Baki Debet',
       'Nilai dalam Mata Uang Asal', 'Kode Kualitas Kredit atau Pembiayaan',
       'Tanggal Macet', 'Kode Sebab Macet', 'Tunggakan Pokok',
       'Tunggakan Bunga atau Imbalan', 'Jumlah Hari Tunggakan',
       'Frekuensi Tunggakan', 'Frekuensi Restrukturisasi',
       'Tanggal Restrukturisasi Awal', 'Tanggal Restrukturisasi Akhir',
       'Kode Cara Restrukturisasi', 'Kode Kondisi', 'Tanggal Kondisi',
       'Keterangan', 'Kode Kantor Cabang', 'Operasi Data', 'Status delete',
       'Create Date', 'Update Date']]
dn3=dn3[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Nomor CIF Lama Debitur', 'Nomor CIF Debitur', 'Jenis Identitas',
       'Nomor Identitas', 'Nama Sesuai Identitas', 'Nama Lengkap',
       'Kode Status Pendidikan atau Gelar Debitur', 'Jenis Kelamin',
       'Tempat Lahir', 'Tanggal Lahir', 'Nama Gadis Ibu Kandung', 'NPWP',
       'Alamat', 'Kelurahan', 'Kecamatan', 'Kode Kabupaten atau Kota',
       'Kode Pos', 'Nomor Telepon', 'Nomor Telepon Seluler', 'Alamat e-mail',
       'Kode Negara Domisili', 'Kode Pekerjaan', 'Tempat Bekerja',
       'Kode Bidang Usaha Tempat Bekerja', 'Alamat Tempat Bekerja',
       'Kode Golongan Debitur', 'Status Perkawinan Debitur',
       'Kode Kantor Cabang', 'Operasi Data', 'Status delete', 'Create Date',
       'Update Date']]

## Export file for KBIJ Reporting

In [258]:
dn3.to_csv(path_or_buf=D01name, sep='|',index =0)
fu3.to_csv(path_or_buf=F01name, sep='|',index =0)

In [108]:
# fu3.to_csv(path_or_buf='F01-201910rev.txt', sep='|',index =0)
# dn3.to_csv(path_or_buf='D01-201910rev.txt', sep='|',index =0)

In [89]:
dn3 = pd.read_csv('D01-201910rev.txt',dtype=str,sep='|')
dn3=dn3[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Nomor CIF Lama Debitur', 'Nomor CIF Debitur', 'Jenis Identitas',
       'Nomor Identitas', 'Nama Sesuai Identitas', 'Nama Lengkap',
       'Kode Status Pendidikan atau Gelar Debitur', 'Jenis Kelamin',
       'Tempat Lahir', 'Tanggal Lahir', 'Nama Gadis Ibu Kandung', 'NPWP',
       'Alamat', 'Kelurahan', 'Kecamatan', 'Kode Kabupaten atau Kota',
       'Kode Pos', 'Nomor Telepon', 'Nomor Telepon Seluler', 'Alamat e-mail',
       'Kode Negara Domisili', 'Kode Pekerjaan', 'Tempat Bekerja',
       'Kode Bidang Usaha Tempat Bekerja', 'Alamat Tempat Bekerja',
       'Kode Golongan Debitur', 'Status Perkawinan Debitur',
       'Kode Kantor Cabang', 'Operasi Data', 'Status delete', 'Create Date',
       'Update Date']]
dn3.to_csv(path_or_buf='D01-201910rev1.txt', sep='|',index =0)

In [55]:
fu3.loc[fu3['Nomor Rekening Fasilitas']=='ATH-11V6ACU6'].head(5)

,Baki Debet,Create Date,Denda,Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan,Frekuensi Restrukturisasi,Frekuensi Tunggakan,Jenis Suku Bunga atau Imbalan,Jumlah Hari Tunggakan,Keterangan,Kode Akad Kredit atau Akad Pembiayaan,...,Tanggal Jatuh Tempo,Tanggal Kondisi,Tanggal Macet,Tanggal Mulai,Tanggal Restrukturisasi Akhir,Tanggal Restrukturisasi Awal,Tunggakan Bunga atau Imbalan,Tunggakan Pokok,Update Date,date
195884,0,20190901065347,0,0,0,0,1,0,NaN,00,...,20191001,20191001,NaN,20190901,NaN,NaN,0,0,20191001123818,2019-10-01 12:38:18


In [54]:
# fu3[['Nomor Rekening Fasilitas']].merge(data,how='inner',on='Nomor Rekening Fasilitas').to_csv('datahilang.csv')
fu3[['Nomor Rekening Fasilitas']].merge(data,how='inner',on='Nomor Rekening Fasilitas').head(5)

,Nomor Rekening Fasilitas,Kode Jenis Pelapor,Kode Pelapor,Tahun Bulan Data,Kode Jenis Fasilitas,Nomor Rekening Lama Fasilitas,Nomor CIF Debitur,Kode Sifat Kredit atau Pembiayaan,Kode Jenis Kredit atau Pembiayaan,Kode Akad Kredit atau Akad Pembiayaan,...,Tanggal Restrukturisasi Akhir,Kode Cara Restrukturisasi,Kode Kondisi,Tanggal Kondisi,Keterangan,Kode Kantor Cabang,Operasi Data,Status delete,Create Date,Update Date
0,ATH-11V6ACU6,0301,900013,201909,F01,NaN,3915986035069,9,05,00,...,NaN,NaN,00,NaN,NaN,000,C,T,20190901065347,20191001123818
1,ATH-12GGG7F8,0301,900013,201909,F01,NaN,3915959152551,9,05,00,...,NaN,NaN,00,NaN,NaN,000,C,T,20190904041922,20191001080137
2,ATH-12KEX5FY,0301,900013,201909,F01,NaN,3915924571121,9,05,00,...,NaN,NaN,00,NaN,NaN,000,C,T,20190903110341,20191003113715
3,ATH-12Q28B4A,0301,900013,201909,F01,NaN,3915928864482,9,05,00,...,NaN,NaN,00,NaN,NaN,000,C,T,20190907071057,20191001014308
4,ATH-132482K6,0301,900013,201909,F01,NaN,3915924558038,9,05,00,...,NaN,NaN,00,NaN,NaN,000,C,T,20190902082805,20191002035453


In [21]:
dn3 = pd.read_csv('D01-201910rev2.txt',dtype=str,sep='|')
dn3=dn3[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Nomor CIF Lama Debitur', 'Nomor CIF Debitur', 'Jenis Identitas',
       'Nomor Identitas', 'Nama Sesuai Identitas', 'Nama Lengkap',
       'Kode Status Pendidikan atau Gelar Debitur', 'Jenis Kelamin',
       'Tempat Lahir', 'Tanggal Lahir', 'Nama Gadis Ibu Kandung', 'NPWP',
       'Alamat', 'Kelurahan', 'Kecamatan', 'Kode Kabupaten atau Kota',
       'Kode Pos', 'Nomor Telepon', 'Nomor Telepon Seluler', 'Alamat e-mail',
       'Kode Negara Domisili', 'Kode Pekerjaan', 'Tempat Bekerja',
       'Kode Bidang Usaha Tempat Bekerja', 'Alamat Tempat Bekerja',
       'Kode Golongan Debitur', 'Status Perkawinan Debitur',
       'Kode Kantor Cabang', 'Operasi Data', 'Status delete', 'Create Date',
       'Update Date']]
print(data.merge(dn3,how='inner',on='Nomor CIF Debitur').shape)
print(data.merge(D01new,how='inner',on='Nomor CIF Debitur').shape)

(1307, 87)
(1515, 87)


In [1]:
F01new = pd.read_csv('F01201910.csv',dtype=str)
F01new.shape

(348107, 54)

In [3]:
F01new['Kode Kualitas Kredit atau Pembiayaan'].value_counts()

1    264433
2     60417
5     12656
4      6759
3      3842
Name: Kode Kualitas Kredit atau Pembiayaan, dtype: int64

In [80]:
# # data F01-201910rev1
# fu3 = pd.read_csv('F01-201910rev.txt',dtype=str,sep='|')
# fu3['Kode Sebab Macet'] = fu3['Kode Sebab Macet'].str.replace('.0', '')
# fu3=fu3[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
#        'Kode Jenis Fasilitas', 'Nomor Rekening Fasilitas',
#        'Nomor Rekening Lama Fasilitas', 'Nomor CIF Debitur',
#        'Kode Sifat Kredit atau Pembiayaan',
#        'Kode Jenis Kredit atau Pembiayaan',
#        'Kode Akad Kredit atau Akad Pembiayaan', 'Nomor Akad Awal',
#        'Tanggal Akad Awal', 'Nomor Akad Akhir', 'Tanggal Akad Akhir',
#        'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan',
#        'Tanggal Awal Kredit atau Pembiayaan', 'Tanggal Mulai',
#        'Tanggal Jatuh Tempo', 'Kode Kategori Debitur', 'Kode Jenis Penggunaan',
#        'Kode Orientasi Penggunaan', 'Kode Sektor Ekonomi',
#        'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan',
#        'Nilai Proyek', 'Kode Valuta', 'Suku Bunga atau Imbalan',
#        'Jenis Suku Bunga atau Imbalan',
#        'Kredit atau Pembiayaan Program Pemerintah', 'Plafon Awal', 'Plafon',
#        'Realisasi atau Pencairan Bulan Berjalan', 'Denda', 'Baki Debet',
#        'Nilai dalam Mata Uang Asal', 'Kode Kualitas Kredit atau Pembiayaan',
#        'Tanggal Macet', 'Kode Sebab Macet', 'Tunggakan Pokok',
#        'Tunggakan Bunga atau Imbalan', 'Jumlah Hari Tunggakan',
#        'Frekuensi Tunggakan', 'Frekuensi Restrukturisasi',
#        'Tanggal Restrukturisasi Awal', 'Tanggal Restrukturisasi Akhir',
#        'Kode Cara Restrukturisasi', 'Kode Kondisi', 'Tanggal Kondisi',
#        'Keterangan', 'Kode Kantor Cabang', 'Operasi Data', 'Status delete',
#        'Create Date', 'Update Date']]
# print(fu3['Kode Sebab Macet'].value_counts())
# fu3.to_csv(path_or_buf='F01-201910rev1.txt', sep='|',index =0)

99    12454
Name: Kode Sebab Macet, dtype: int64


In [148]:
# print(fu3.shape,datahilang.shape)
# fu3.columns==datahilang.columns
# final = pd.concat([fu3,datahilang])
# final.to_csv(path_or_buf='F01-201910rev1complete.txt', sep='|',index =0)
# fu3 = final

Export full data dump to csv to be used in the next iteration

In [260]:
F01new.to_csv(path_or_buf='F01201910.csv', index=0)
# D01new.to_csv(path_or_buf='F01201909.csv', index=0)
D01new.to_csv(path_or_buf='D01201910.csv', index=0)

### Additional Data 201910

In [20]:
F01new = pd.read_csv('F01201910.csv',dtype=str)
print(F01new.shape)
print(pd.read_csv('F01-201910rev1additional.txt',sep='|',dtype=str).shape)

(348107, 54)
(1515, 53)


In [21]:
a = pd.read_csv('missingdata201910.csv')
print(a.shape)
a = a.merge(F01new, how='inner', left_on='Nomor Rekening Fasilitas 201909',right_on='Nomor Rekening Fasilitas')
a['Operasi Data']='U'
a['Tahun Bulan Data']='201910'
a = a[['Kode Jenis Pelapor', 'Kode Pelapor', 'Tahun Bulan Data',
       'Kode Jenis Fasilitas', 'Nomor Rekening Fasilitas',
       'Nomor Rekening Lama Fasilitas', 'Nomor CIF Debitur',
       'Kode Sifat Kredit atau Pembiayaan',
       'Kode Jenis Kredit atau Pembiayaan',
       'Kode Akad Kredit atau Akad Pembiayaan', 'Nomor Akad Awal',
       'Tanggal Akad Awal', 'Nomor Akad Akhir', 'Tanggal Akad Akhir',
       'Frekuensi Perpanjangan Fasilitas Kredit atau Pembiayaan',
       'Tanggal Awal Kredit atau Pembiayaan', 'Tanggal Mulai',
       'Tanggal Jatuh Tempo', 'Kode Kategori Debitur', 'Kode Jenis Penggunaan',
       'Kode Orientasi Penggunaan', 'Kode Sektor Ekonomi',
       'Kode Kabupaten atau Kota Lokasi Proyek atau Penggunaan Kredit atau Pembiayaan',
       'Nilai Proyek', 'Kode Valuta', 'Suku Bunga atau Imbalan',
       'Jenis Suku Bunga atau Imbalan',
       'Kredit atau Pembiayaan Program Pemerintah', 'Plafon Awal', 'Plafon',
       'Realisasi atau Pencairan Bulan Berjalan', 'Denda', 'Baki Debet',
       'Nilai dalam Mata Uang Asal', 'Kode Kualitas Kredit atau Pembiayaan',
       'Tanggal Macet', 'Kode Sebab Macet', 'Tunggakan Pokok',
       'Tunggakan Bunga atau Imbalan', 'Jumlah Hari Tunggakan',
       'Frekuensi Tunggakan', 'Frekuensi Restrukturisasi',
       'Tanggal Restrukturisasi Awal', 'Tanggal Restrukturisasi Akhir',
       'Kode Cara Restrukturisasi', 'Kode Kondisi', 'Tanggal Kondisi',
       'Keterangan', 'Kode Kantor Cabang', 'Operasi Data', 'Status delete',
       'Create Date', 'Update Date']]
a.to_csv(path_or_buf='F01-201910rev2additional.txt', sep='|',index =0)
print(a.shape)

(2786, 1)
(2786, 53)


In [22]:
# a.loc[a['Nomor Rekening Fasilitas']=='ATH-SL5GH4NS'].head()

In [23]:
# x = pd.read_csv('F01-201909.txt',sep='|',dtype=str)
# x.loc[x['Nomor Rekening Fasilitas']=='ATH-SL5GH4NS'].head()